%store -r main_name
%store -r TT
%store -r V_avg
%store -r N
%store -r P
%store -r Walk
%store -r NC_upper
%store -r NC_lower

In [ ]:
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition
import sympy as sym
import numpy as np
from IPython.display import display
from sympy import pprint
import dill
dill.load_session('EOM_V3.db')


def hide_sloution(hiddenLines):
    from IPython.core.display import display, HTML
    toggle_code_str = '''
    <form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Solution: {0}"></form>
    '''.format(hiddenLines)

    toggle_code_prepare_str = '''
        <script>
        function code_toggle() {
            if ($('div.cell.code_cell.rendered.selected div.input').css('display')!='none'){
                $('div.cell.code_cell.rendered.selected div.input').hide();
            } else {
                $('div.cell.code_cell.rendered.selected div.input').show();
            }y
            
        }
        </script>

    '''

    display(HTML(toggle_code_prepare_str))
    display(HTML(toggle_code_str))
    
hide_sloution('Import Files')

In [ ]:
hide_sloution('Symbolic Imports and Parameters')
func_map = {'sin':sin, 'cos':cos} 

sym_list = [g] + [N_GR] + [yBO] + [xBO] + [lx_boom_bodyCOM] + [ly_boom_bodyCOM_y]+\
           [lx_boom] + [lx_boom_COM] + [ly_boom] + [ly_boom_COM] + [y_boomOffset]+\
           [foot_length] + [foot_angle] + [angle_offset] + [K_spring] + [B_damper] +\
           [masses[base] for base in bases] +\
           [masses[link,leg] for leg in legs for link in links] +\
           [lengths[base] for base in bases] +\
           [lengths[link,leg] for leg in legs for link in links] +\
           [lengths_COM[link,leg] for leg in legs for link in links] +\
           [inertias['body']] + [inertias['boom','X']] + [inertias['boom','Y']] + [inertias['motors']] +\
           [inertias[link,leg] for leg in legs for link in links] +\
           [xb]+[yb]+[th[link,leg] for leg in legs for link in links] +\
           [dxb]+[dyb]+[dth[link,leg] for leg in legs for link in links] +\
           [ddxb]+[ddyb]+[ddth[link,leg] for leg in legs for link in links] +\
           [tau['left',leg] for leg in legs] + [tau['right',leg] for leg in legs] +\
           [lamda['x',leg] for leg in legs] + [lamda['y',leg] for leg in legs] +\
           [GRF['x',leg] for leg in legs] + [GRF['y',leg] for leg in legs]

lamb_EOMs = {}

DOFs = ['xb'] + ['yb']

for leg in legs:
    for link in links:
        DOFs = DOFs + ['theta_{0}'.format(link+leg)]

for dof_i,dof in enumerate(DOFs):
    lamb_EOMs.update({dof: sym.lambdify(sym_list,EOMs[dof_i],modules = [func_map])})

# PARAMETERS:

lengths_COM_ = {'ULA': 41.14/1000,
                 'LLA': 132.2/1000,
                 'URA': 41.14/1000,
                 'LRA': sqrt(190.4**2 + 1.63**2)/1000,
                 'sfootA': 0.2/1000,
                 'ULB': 41.14/1000,
                 'LLB': 132.2/1000,
                 'URB': 41.14/1000,         
                 'LRB': sqrt(190.4**2 + 1.63**2)/1000,
                 'sfootB': 0.2/1000}

masses_ = {'body': 5756.78/1000,
           'boom': 2470.66/1000,
           'ULA': 153.81/1000,
           'LLA': 346.45/1000,
           'URA': 153.81/1000,
           'LRA': 499.95/1000,
           'sfootA': 13.53/1000,
           'ULB': 153.81/1000,
           'LLB': 346.45/1000,
           'URB': 153.81/1000,
           'LRB': 499.95/1000,
           'sfootB': 13.53/1000}

lengths_ = {'body': 253/1000,
            'boom': 2.5/1000,
            'ULA': 174.5/1000,
            'LLA': 300.0/1000,
            'URA': 174.5/1000,
            'LRA': 295.5/1000,
            'sfootA': 0.4/1000,
            'ULB': 174.5/1000,
            'LLB': 300.0/1000,
            'URB': 174.5/1000,
            'LRB': 295.5/1000,
            'sfootB': 0.4/1000}

##### QUERY THIS????????
inertias_ = {'body': 32124078.52*1e-9, # body rotate about its center 
             'boom_X': 1947254622.19*1e-9,
             'boom_Y': 1947254622.19*1e-9,
             'motors': 181877.41*1e-9,
             'ULA': 600933.68*1e-9, # rotates as pendulumn
             'LLA': 4659658.63*1e-9,
             'URA': 600933.68*1e-9,
             'LRA': 7369968.50*1e-9,
             'sfootA': 0.0,
             'ULB': 600933.68*1e-9, # rotates as pendulumn 
             'LLB': 4659658.63*1e-9,
             'URB': 600933.68*1e-9,
             'LRB': 7369968.50*1e-9,
             'sfootB': 0.0}

all_links = []
all_links_inertia = []
leg_links = []
[leg_links.append(link+leg) for leg in legs for link in links]
[all_links.append(base) for base in bases]
[all_links.append(link+leg) for leg in legs for link in links]
all_links_inertia.extend(['body','boom_X','boom_Y','motors'])
[all_links_inertia.append(link+leg) for leg in legs for link in links]


total_mass = 0.0
for mass in masses_: # must equal ~ 9.1 - 9.5kg
    if mass == 'boom':
        total_mass += masses_[mass]*0.5
    else:
        total_mass += masses_[mass]
total_mass = round(total_mass,2)
print("Mass must equal 9.1 - 9.5kg, current mass value = ",total_mass)


WDOFs = ['X','Y'] 
signs = ['-ve','+ve'] 
#ground_constraints = ['contact','friction','slip_+ve','slip_-ve','no_slip'] # Group all constraints occuring at the ground
# ground_constraints = ['contact','no_slip','footSpring'] 
ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
sides = ['left','right']
coordinates = {'X':0 , 'Y':1}
motor_w_angle = {'left': 'UL' ,'right': 'UR'}

def get_var_list_zeros(m,n,p):
    var_list = [m.g] + [m.N_GR] + [m.yBO] + [m.xBO] +[m.lx_boom_bodyCOM] + [m.ly_boom_bodyCOM_y]+\
               [m.lx_boom] + [m.lx_boom_COM] + [m.ly_boom] + [m.ly_boom_COM] + [m.y_boomOffset]+\
               [m.foot_length] + [m.foot_angle] + [m.angle_offset]+ [m.K_spring] + [m.B_damper] +\
               [m.masses[base] for base in bases] +\
               [m.masses[link+leg] for leg in legs for link in links] +\
               [m.lengths[base] for base in bases] +\
               [m.lengths[link+leg] for leg in legs for link in links] +\
               [m.lengths_COM[link+leg] for leg in legs for link in links] +\
               [m.inertias['body']] + [m.inertias['boom_X']] + [m.inertias['boom_Y']] + [m.inertias['motors']]+\
               [m.inertias[link+leg] for leg in legs for link in links] +\
               [m.q[n,p,dof] for dof in DOFs] +\
               [m.dq[n,p,dof] for dof in DOFs] +\
               [m.ddq[n,p,dof] for dof in DOFs] +\
               [0 for leg in legs] + [0 for leg in legs] +\
               [0 for leg in legs] + [0 for leg in legs] +\
               [0 for leg in legs] + [0 for leg in legs]
    return var_list

def get_var_list_values(m,n,p):
    
    BW = 1
    tau_value = {}
    connection_force_value = {}
    GRF_X_value = {}
    GRF_Y_value = {}

    for leg in legs:
#             [tau_value.update({(side,leg): BW*(m.tau_torque[n,p,side,leg])}) for side in sides]
#             [connection_force_value.update({(wdof,leg):BW*(m.connect_force[n,p,wdof,leg])}) for wdof in WDOFs]
#             GRF_X_value.update({(leg):BW*(m.GRF_x[n,p,'+ve',leg]-m.GRF_x[n,p,'-ve',leg])})
#             GRF_Y_value.update({(leg):BW*(m.GRF_y[n,p,leg])})


        [tau_value.update({(side,leg): BW*(m.tau_torque[n,side,leg])}) for side in sides]
        [connection_force_value.update({(wdof,leg):BW*(m.connect_force[n,p,wdof,leg])}) for wdof in WDOFs]
        GRF_X_value.update({(leg):BW*(m.GRF_x[n,p,'+ve',leg]-m.GRF_x[n,p,'-ve',leg])})
        GRF_Y_value.update({(leg):BW*(m.GRF_y[n,p,leg])})

    var_list = [m.g] + [m.N_GR] + [m.yBO] + [m.xBO] +[m.lx_boom_bodyCOM] + [m.ly_boom_bodyCOM_y]+\
               [m.lx_boom] + [m.lx_boom_COM] + [m.ly_boom] + [m.ly_boom_COM] + [m.y_boomOffset]+\
               [m.foot_length] + [m.foot_angle] + [m.angle_offset] + [m.K_spring] + [m.B_damper]+\
               [m.masses[base] for base in bases] +\
               [m.masses[link+leg] for leg in legs for link in links] +\
               [m.lengths[base] for base in bases] +\
               [m.lengths[link+leg] for leg in legs for link in links] +\
               [m.lengths_COM[link+leg] for leg in legs for link in links] +\
               [m.inertias['body']] + [m.inertias['boom_X']] + [m.inertias['boom_Y']] + [m.inertias['motors']]+\
               [m.inertias[link+leg] for leg in legs for link in links] +\
               [m.q[n,p,dof] for dof in DOFs] +\
               [m.dq[n,p,dof] for dof in DOFs] +\
               [m.ddq[n,p,dof] for dof in DOFs] +\
               [tau_value['left',leg] for leg in legs] + [tau_value['right',leg] for leg in legs] +\
               [connection_force_value['X',leg] for leg in legs] + [connection_force_value['Y',leg] for leg in legs] +\
               [GRF_X_value[leg] for leg in legs] + [GRF_Y_value[leg] for leg in legs]
    return var_list



In [ ]:
hide_sloution('Test selection')

# NOTE: there is a link between the knee angle and unstretched leg length - kinematics
# knee = 120deg --> r = 0.45
#unstretched_leg_length = 0.38
# speed_factor = 2.6/2 # Remy paper 0.4, 0.5 and 1
# V_avg = speed_factor*sqrt(unstretched_leg_length*9.81)

tests = {0:'High_Drop', 1:'Low_Drop', 2:'Hop', 3:'Periodic_Walk', 4: 'Periodic_Run', 5: 'check'}

if Walk == True:
    test = tests[3]
else:
    test = tests[4]

# Set parameters
h_min = 0.5 #0.8
h_max = 1.2 #1.2
knee_angle_max = 160 # deg - knee extention
knee_angle_min = 20 # deg - knee flexion
sfoot_max = 0.01
sfoot_min = -sfoot_max
µ = 0.8 # if µ = 1 then it will slip but if µ<0.5 then it will knock its foot hard into the ground.

# TT: want it to be low as it use less tau sum but not too low as the freq of motion will be too fast

        
m = ConcreteModel('Baleka2_Radau')

In [ ]:
hide_sloution('Connection Constraint and Constant Sets')

#SETS-------------------------
m.N = RangeSet(N)
m.P = Set(initialize = range(P+1)) # collocation points
m.ground_constraints = Set(initialize = ground_constraints)
m.legs = Set(initialize = legs)
m.links = Set(initialize = links)
m.bases = Set(initialize = bases)
m.all_links = Set(initialize = all_links)
m.leg_links = Set(initialize = leg_links)
m.all_links_inertia = Set(initialize = all_links_inertia)
m.DOFs = Set(initialize = DOFs)
m.WDOFs = Set(initialize = WDOFs)
m.signs = Set(initialize = signs)
m.sides = Set(initialize = sides)

m.g = Param(initialize = 9.81)
m.µ = Param(initialize = µ)
m.K_spring = Param(initialize = 1e4)
m.B_damper = Param(initialize = -1e3)
    
m.N_GR = Param(initialize = 9)
m.yBO = Param(initialize = 8.36/1000)
m.xBO = Param(initialize = 60/1000)
m.lx_boom = Param(initialize = 2.575)
m.ly_boom = Param(initialize = 2.493)
m.lx_boom_COM = Param(initialize = 1.149)
m.ly_boom_COM = Param(initialize = 1.149)
m.lx_boom_bodyCOM = Param(initialize = 0.0)
m.ly_boom_bodyCOM_y = Param(initialize = 0.0)

y_mat_height = 0.0 # since the mat is not under the boom as well.
m.y_boomOffset = Param(initialize = 0.101-y_mat_height)
m.foot_length = Param(initialize = 42.5/1000)
m.foot_angle = Param(initialize = 135*(np.pi/180))
m.angle_offset = Param(initialize = 0.0085607152371996) # sym.atan(1.63/190.4) - from COM link to actual connect position

m.lengths_COM = Param(m.leg_links, initialize = lengths_COM_)
m.masses = Param(m.all_links, initialize = masses_)
m.lengths = Param(m.all_links, initialize = lengths_)
m.inertias = Param(m.all_links_inertia, initialize = inertias_)

In [ ]:
hide_sloution('Radau')
# m.h[1].fix(0.0)


#VARIABLES----------------------
#gen coordinates
m.q = Var(m.N,m.P,m.DOFs)
m.dq = Var(m.N,m.P,m.DOFs)
m.ddq = Var(m.N,m.P,m.DOFs)

''' CONSTRAINTS  - Integration'''
# variable timestep
hm = TT/N # master timestep
m.h = Var(m.N, bounds = (h_min,h_max)) #m.h = Var(m.N, bounds = (0.5,1.5))

# Integration constraints 
# https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjbwNzHpNeDAxVRZ0EAHbkxAgEQFnoECBIQAQ&url=https%3A%2F%2Fwww.researchgate.net%2Fpublication%2F302467919_Radau_Methods&usg=AOvVaw3fxR9UT_1h6j4XktAeL2bo&opi=89978449
col_mat = [[0.41666667, -0.08333333],
           [0.75, 0.25]]


def integrate_p(m,n,p,dof):
    if p == 0: # continuity constraint:
        if n == 1: return Constraint.Skip
        return m.q[n,0,dof] == m.q[n-1,P,dof]
    if p > 0:
        return m.q[n,p,dof] == m.q[n,0,dof] + hm*m.h[n]*sum([col_mat[p-1][pp-1]*m.dq[n,pp,dof] for pp in range(1,P+1)])
m.integrate_p = Constraint(m.N, m.P, m.DOFs, rule = integrate_p)

def integrate_v(m,n,p,dof):
    if p == 0: # continuity constraint:
        if n == 1: return Constraint.Skip
        return m.dq[n,0,dof] == m.dq[n-1,P,dof]
    if p > 0:
        return m.dq[n,p,dof] == m.dq[n,0,dof] + hm*m.h[n]*sum([col_mat[p-1][pp-1]*m.ddq[n,pp,dof] for pp in range(1,P+1)])
m.integrate_v = Constraint(m.N, m.P, m.DOFs, rule = integrate_v)


''' CONSTRAINTS  - CONNECTION POINT'''
m.connect_position = Var(m.N, m.P, m.WDOFs, m.sides, m.legs) # connection point
m.connect_force = Var(m.N, m.P, m.WDOFs, m.legs,bounds = (-10*total_mass*9.81,10*total_mass*9.81)) # connection force

# Lamdify
lamb_connect_position = {} # (WDOFs,sides,leg)
[lamb_connect_position.update({(wdof,side,leg): sym.lambdify(sym_list,connect_position[side,leg][coordinates[wdof]],modules = [func_map])}) for leg in legs for side in sides for wdof in WDOFs]

# Equate the connection equations from the previous script to the model m
def def_connect_position_equation(m,n,p,wdof,side,leg):
    if p == 0 and n > 1:
        return m.connect_position[n,0,wdof,side,leg] == m.connect_position[n-1,P,wdof,side,leg]
    else:
        var_list = get_var_list_zeros(m,n,p) 
        return m.connect_position[n,p,wdof,side,leg] == lamb_connect_position[wdof,side,leg](*var_list)
m.connect_position_equation = Constraint(m.N, m.P, m.WDOFs, m.sides, m.legs, rule = def_connect_position_equation)

# Ensure that there is no distance between the X (and Y) of link connection points
def def_connect_position(m,n,p,wdof,leg,side):
    if p == 0:
        if n == 1: return Constraint.Skip
        if side == 'right':
            return m.connect_position[n,0,wdof,'right',leg] == m.connect_position[n-1,P,wdof,'right',leg]
        else:
            return m.connect_position[n,0,wdof,'left',leg] == m.connect_position[n-1,P,wdof,'left',leg]
    return  m.connect_position[n,p,wdof,'right',leg] - m.connect_position[n,p,wdof,'left',leg] == 0.0
m.connect_position_constraint = Constraint(m.N, m.P, m.WDOFs, m.legs, m.sides, rule= def_connect_position)

In [ ]:
hide_sloution('Torque Speed Curve Display')
#NOTE THIS MUST BE CHANGED IF THBODY IS INCLUDED

voltage = 24

if voltage == 24:
    no_load_speed = 23.2 #round(24*(100/9) * ((2*np.pi)/60),1)
    speed_rated_torque = 15.4 #round(222* ((2*np.pi)/60),1)
else:
    no_load_speed = round(48*(100/9) * ((2*np.pi)/60),1)
    speed_rated_torque = round(445* ((2*np.pi)/60),1)    

rated_torque = 18.0
peak_torque = 54.0
zero_offset = 2.23
desired_max_torque = 30.0

'''Plot Torque Speed Curve'''
import matplotlib.pyplot as plt
x_p = np.linspace(0, int(no_load_speed+1), num=int(no_load_speed+1))
x_n = np.linspace(-int(no_load_speed+1), 0, num=int(no_load_speed+1))

m_grad = (rated_torque-0.0)/(speed_rated_torque-no_load_speed)
stall_torque = 0.0 - m_grad*no_load_speed

y_p = m_grad*x_p+stall_torque
y_n = m_grad*x_n-stall_torque

stall_torque_p = []
stall_torque_n = []
rated_torque_p = []
rated_torque_n = []

for ii in range(len(x_p)):
    stall_torque_p.append(stall_torque)
    stall_torque_n.append(-stall_torque)
    rated_torque_p.append(peak_torque)
    rated_torque_n.append(-peak_torque)

plt.grid()
plt.plot(x_p, y_p,'b')
plt.plot(x_n, stall_torque_p,'b')

plt.fill_between(x_p, y_p, alpha=0.4, color = 'cyan')
plt.fill_between(x_n, stall_torque_p, alpha=0.4,color = 'cyan')

plt.plot(x_n, y_n,'r')
plt.plot(x_p, stall_torque_n,'r')

plt.fill_between(x_n, y_n, alpha=0.4,color = 'pink')
plt.fill_between(x_p, stall_torque_n, alpha=0.4,color = 'pink')

plt.axis((-30,30,-90,90))
plt.title('New Torque Speed Curve')
plt.xlabel('Speed (rad/s)')
plt.ylabel('Torque (Nm)')
print('--------------')

print('Stall torque = ',stall_torque)
print('Max speed = ',no_load_speed)

print('--------------')

#m.tau_torque = Var(m.N, m.sides, m.legs, bounds = (-peak_torque,peak_torque))
m.tau_torque = Var(m.N, m.sides, m.legs, bounds = (-desired_max_torque,desired_max_torque))
m.motor_velocity = Var(m.N, m.sides, m.legs, m.signs, bounds = (0.0,no_load_speed))

# for n in range (1,1+N):
#     for dof_i in DOFs:
#         if 'U' in dof_i:
#             # if it is an upper link
#             m.dq[n,dof_i].setub(no_load_speed)
#             m.dq[n,dof_i].setlb(-no_load_speed)
#         else:
#             pass

# CONSTRAINTS
def def_motor_velocity(m,n,p,dof_i):
    if p>0: return Constraint.Skip #since the current node equates the previous cp
    else:
        if 'U' in dof_i:
            leg = dof_i[-1]
            if 'L' in dof_i:
                side = 'left'
            else:
                side = 'right'
            return m.motor_velocity[n,side,leg,'+ve']-m.motor_velocity[n,side,leg,'-ve'] == m.dq[n,p,dof_i]
        else:
            return Constraint.Skip
m.def_motor_velocity_constraint = Constraint(m.N, m.P, m.DOFs, rule = def_motor_velocity)

# assume straight line
def def_TW(m,n,side,leg,sign):
    if sign == '+ve':
        return m.tau_torque[n,side,leg] <= m.motor_velocity[n,side,leg,'+ve']*m_grad + stall_torque
    else:
        return m.tau_torque[n,side,leg] >= -m.motor_velocity[n,side,leg,'-ve']*m_grad - stall_torque
m.TW_constraint = Constraint(m.N, m.sides, m.legs, m.signs, rule = def_TW)


# # Same tau across each collocation point
# def def_TorqueConstraintv1(m,n,side,leg,sign):
#     return m.tau_torque[n,1,side,leg] == m.tau_torque[n,2,side,leg]
# m.TorqueConstraintv1_constraint = Constraint(m.N, m.sides, m.legs, m.signs, rule = def_TorqueConstraintv1)



In [ ]:
hide_sloution('Ground Contact Constraints')
# # Variables
m.foot_position = Var(m.N, m.P, m.WDOFs, m.legs)
m.foot_velocity = Var(m.N, m.P, m.WDOFs, m.signs, m.legs, bounds = (0.0,None)) # the sign will account for -ve velocities
m.foot_velocity_x = Var(m.N, m.P, m.legs) 
m.foot_velocity_y = Var(m.N, m.P, m.legs) 
m.friction_cone = Var(m.N, m.P, m.legs)

lim_GRF = 10*total_mass*m.g.value
m.GRF_y = Var(m.N, m.P, m.legs, bounds = (0.0,lim_GRF)) # ground reaction forces
m.GRF_x = Var(m.N, m.P, m.signs, m.legs, bounds = (0.0,lim_GRF)) # ground reaction forces
m.ground_penalty = Var(m.N, m.ground_constraints, m.legs, bounds = (0.0,10.0)) # penalty

# #Get the foot position in WDOF
lamb_foot_position = {}
lamb_foot_velocity_x = {}
lamb_foot_velocity_y = {}

[lamb_foot_position.update({(wdof,leg): sym.lambdify(sym_list,foot_position[leg][coordinates[wdof]],modules = [func_map])}) for leg in legs for wdof in WDOFs]
[lamb_foot_velocity_x.update({(leg): sym.lambdify(sym_list,(foot_position[leg].jacobian(q)*dq)[0].simplify(),modules = [func_map])}) for leg in legs]
[lamb_foot_velocity_y.update({(leg): sym.lambdify(sym_list,(foot_position[leg].jacobian(q)*dq)[1].simplify(),modules = [func_map])}) for leg in legs]

def def_foot_position(m,n,p,wdof,leg):
    if p == 0 and n > 1:
        return m.foot_position[n,0,wdof,leg] == m.foot_position[n-1,P,wdof,leg]
    else:
        var_list = get_var_list_zeros(m,n,p)
        return m.foot_position[n,p,wdof,leg] == lamb_foot_position[wdof,leg](*var_list)
m.def_foot_position = Constraint(m.N, m.P, m.WDOFs, m.legs, rule = def_foot_position)

# CONSTRAINTS
def def_foot_velocity(m,n,p,wdof,leg, sgn):
    if wdof == 'X':
        if p == 0 and n > 1:
            if sgn == '+ve':
                return m.foot_velocity[n,0,'X','+ve',leg] == m.foot_velocity[n-1,P,'X','+ve',leg]
            else:
                return m.foot_velocity[n,0,'X','-ve',leg] == m.foot_velocity[n-1,P,'X','-ve',leg]
            
        var_list = get_var_list_zeros(m,n,p)
        return m.foot_velocity[n,p,'X','+ve',leg]-m.foot_velocity[n,p,'X','-ve',leg] == lamb_foot_velocity_x[leg](*var_list)
   
    else: 
        if p == 0 and n > 1:
            if sgn == '+ve':
                return m.foot_velocity[n,0,'Y','+ve',leg] == m.foot_velocity[n-1,P,'Y','+ve',leg]
            else:
                return m.foot_velocity[n,0,'Y','-ve',leg] == m.foot_velocity[n-1,P,'Y','-ve',leg]
            
        var_list = get_var_list_zeros(m,n,p)
        return m.foot_velocity[n,p,'Y','+ve',leg]-m.foot_velocity[n,p,'Y','-ve',leg] == lamb_foot_velocity_y[leg](*var_list)
m.foot_velocity_constraint = Constraint(m.N, m.P, m.WDOFs, m.legs, m.signs, rule = def_foot_velocity)

def def_foot_velocity_xy(m,n,p,wdof,leg):
    var_list = get_var_list_zeros(m,n,p)
    if wdof == 'X':
        if p == 0 and n > 1:
            return m.foot_velocity_x[n,0,leg]  == m.foot_velocity_x[n-1,P,leg] 
        return m.foot_velocity_x[n,p,leg] == lamb_foot_velocity_x[leg](*var_list)
    else:
        if p == 0 and n > 1:
            return m.foot_velocity_y[n,0,leg] == m.foot_velocity_y[n-1,P,leg]
        return m.foot_velocity_y[n,p,leg] == lamb_foot_velocity_y[leg](*var_list)
m.def_foot_velocity_xy_constraint = Constraint(m.N, m.P, m.WDOFs, m.legs, rule = def_foot_velocity_xy)

m.ground_varA = Var(m.N, m.ground_constraints,m.legs)
m.ground_varB = Var(m.N, m.ground_constraints,m.legs)

# GRFx = µ*GRFy (stick to ground - static friction)
def def_friction_cone(m,n,p,leg):
    if p == 0: return Constraint.Skip
#     return m.friction_cone[n,p,leg] <= m.µ*m.GRF_y[n,p,leg] - (m.GRF_x[n,p,'+ve',leg] + m.GRF_x[n,p,'-ve',leg])
    return m.µ*m.GRF_y[n,p,leg] - (m.GRF_x[n,p,'+ve',leg] + m.GRF_x[n,p,'-ve',leg]) >= 0.0
m.def_friction_cone_constraint = Constraint(m.N, m.P, m.legs, rule = def_friction_cone)


def def_ground_varA(m,n,gc,leg):
#     if n == 1: return Constraint.Skip
    if gc == 'contact':
        if n < N:
            return m.ground_varA[n,gc,leg] == sum(m.foot_position[n+1,p,'Y',leg] for p in range(P+1))
        else: 
            return m.ground_varA[n,gc,leg] == sum(m.foot_position[n,p,'Y',leg] for p in range(P+1))
    if gc == 'friction':
        return m.ground_varA[n,gc,leg] == sum(m.foot_velocity[n,p,'X','+ve',leg]+m.foot_velocity[n,p,'X','-ve',leg] for p in range(P+1))
    if gc == 'slip_ps':
        return m.ground_varA[n,gc,leg] == sum(m.foot_velocity[n,p,'X','+ve',leg] for p in range(P+1))
    if gc == 'slip_ng':
        return m.ground_varA[n,gc,leg] == sum(m.foot_velocity[n,p,'X','-ve',leg] for p in range(P+1))
    
m.def_ground_varA = Constraint(m.N, m.ground_constraints, m.legs, rule = def_ground_varA)

def def_ground_varB(m,n,gc,leg):
#     if n == 1: return Constraint.Skip
    if gc == 'contact':
        return m.ground_varB[n,gc,leg] == sum(m.GRF_y[n,p,leg] for p in range(P+1))
    if gc == 'friction':
        return m.ground_varB[n,gc,leg] == sum( (m.µ*m.GRF_y[n,p,leg] - (m.GRF_x[n,p,'+ve',leg] + m.GRF_x[n,p,'-ve',leg])) for p in range(P+1))
    if gc == 'slip_ps':
        return m.ground_varB[n,gc,leg] == sum(m.GRF_x[n,p,'+ve',leg] for p in range(P+1))
    if gc == 'slip_ng':
        return m.ground_varB[n,gc,leg] == sum(m.GRF_x[n,p,'-ve',leg] for p in range(P+1))
m.def_ground_varB = Constraint(m.N, m.ground_constraints, m.legs, rule = def_ground_varB)

# complementarity
def ground_comp(m,n,gc,leg):
#     if p == 0: return Constraint.Skip
    return  m.ground_varA[n,gc,leg]*m.ground_varB[n,gc,leg] <= m.ground_penalty[n,gc,leg]
m.ground_comp = Constraint(m.N, m.ground_constraints, m.legs, rule = ground_comp)



def def_body_foot(m,n,p,leg):
    if p == 0:
        if n == 1: return Constraint.Skip
        return m.foot_position[n,0,'Y',leg] == m.foot_position[n-1,P,'Y',leg]
    return m.foot_position[n,p,'Y',leg] <= m.q[n,p,'yb']
m.def_body_footconstraint = Constraint(m.N, m.P, m.legs, rule = def_body_foot)



for n in range(1,N+1):
    for p in range(P+1):
        for leg in legs:
            m.foot_position[n,p,'Y',leg].setlb(0.0)
        
# for leg in legs:
#     m.GRF_y[N,P,leg].fix(0.0)
#     for sgn in signs:
#         m.GRF_x[N,P,sgn,leg].fix(0.0)

for leg in legs:
#     m.GRF_y[1,0,leg].fix(0.0)
    m.GRF_y[N,P,leg].fix(0.0)
    for sgn in signs:
#         m.GRF_x[1,0,sgn,leg].fix(0.0)
        m.GRF_x[N,P,sgn,leg].fix(0.0)

In [ ]:
hide_sloution('Knee Angle Limit and Periodic Constraints')
# Set the angles for the knees - only use this when actuated 
# This ensures links does not cross
m.knee_angle = Var(m.N, m.P, m.sides, m.legs, bounds = (knee_angle_min*(np.pi)/180,knee_angle_max*(np.pi)/180))

# # RELATIVE
# def def_knee_angle(m,n,side,leg):
#     if side == 'left':
#         return m.knee_angle[n,side,leg] == 2*np.pi - m.q[n,'theta_LL'+leg]
#     else:
#         return m.knee_angle[n,side,leg] == m.q[n,'theta_LR'+leg]   
# m.knee_angle_constraint = Constraint(m.N, m.sides, m.legs, rule = def_knee_angle)

# ABSOLUTE
def def_knee_angle(m,n,p,side,leg):
    if p == 0:
        if n == 1: return Constraint.Skip
        return m.knee_angle[n,0,side,leg] == m.knee_angle[n-1,P,side,leg]
    if side == 'left':      
        return m.knee_angle[n,p,side,leg] == np.pi - m.q[n,p,'theta_LL'+leg] + m.q[n,p,'theta_UL'+leg]
    else:
        return m.knee_angle[n,p,side,leg] == m.q[n,p,'theta_LR'+leg] + np.pi - m.q[n,p,'theta_UR'+leg]    
m.knee_angle_constraint = Constraint(m.N, m.P, m.sides, m.legs, rule = def_knee_angle)

sign_symbols = ['+','-']
boundaries = ['setub','setlb']

for n in range(1,N+1):
    for p in range (P+1):
        for leg in legs:
            for index in range(len(boundaries)):
                eval('m.q[n,p,\'theta_UL\'+leg].{0}(np.pi*(0.5{1}1))'.format(boundaries[index],sign_symbols[index]))
                #eval('m.q[n,\'theta_LL\'+leg].{0}(np.pi*(1{1}1/2))'.format(boundaries[index],sign_symbols[index]))  
                eval('m.q[n,p,\'theta_UR\'+leg].{0}(np.pi*(0.5{1}1))'.format(boundaries[index],sign_symbols[index]))    
                #eval('m.q[n,\'theta_LR\'+leg].{0}(np.pi*(1{1}1/2))'.format(boundaries[index],sign_symbols[index]))



# # having too many constraints reduce optimal results and limits natural motion
# def def_body_center(m,leg):
#     return m.q[1,'xb'] == (m.foot_position[1,'X','A']+m.foot_position[1,'X','B'])/2
# m.def_body_center_constraint = Constraint(rule = def_body_center)



In [ ]:
hide_sloution('Periodic Constraints')
# Get the foot position in WDOF
if "Periodic" in test:
    
    lamb_body_position_x = sym.lambdify(sym_list,(r_bases['body'])[0].simplify(),modules = [func_map])
    m.body_position = Var(m.N, m.P, m.WDOFs) # the sign will account for -ve velocities

    # CONSTRAINTS   
    
    if test == "Periodic_Walk" or test == "Periodic_Run":
        
        # Average velocities
        def def_body_position_average(m,wdof):
            if wdof == 'X':
                return (m.q[N,0,'xb'] - m.q[1,0,'xb'])/(sum(hm*m.h[n] for n in range(1,N+1))) == V_avg
            else: return Constraint.Skip
        m.def_body_position_average_constraint = Constraint(m.WDOFs, rule = def_body_position_average)
    
    if test == "Periodic_Walk" or test == "Periodic_Air" or test == "Periodic_Run" or "Periodic_Hop":  
        # Equate states
        def def_equate_position_states(m,dof_i):
            if dof_i == 'yb' or dof_i == 'theta_body':
                return m.q[1,0,dof_i] == m.q[N,0,dof_i]
            
            elif dof_i == 'theta_ULA':
                return m.q[1,0,'theta_ULA'] == m.q[N,0,'theta_ULB']
            elif dof_i == 'theta_ULB':
                return m.q[1,0,'theta_ULB'] == m.q[N,0,'theta_ULA']
            
            elif dof_i == 'theta_URA':
                return m.q[1,0,'theta_URA'] == m.q[N,0,'theta_URB']
            elif dof_i == 'theta_URB':
                return m.q[1,0,'theta_URB'] == m.q[N,0,'theta_URA']
            
            elif dof_i == 'theta_LLA':
                return m.q[1,0,'theta_LLA'] == m.q[N,0,'theta_LLB']
            elif dof_i == 'theta_LLB':
                return m.q[1,0,'theta_LLB'] == m.q[N,0,'theta_LLA']
            
            elif dof_i == 'theta_LRA':
                return m.q[1,0,'theta_LRA'] == m.q[N,0,'theta_LRB']
            elif dof_i == 'theta_LRB':
                return m.q[1,0,'theta_LRB'] == m.q[N,0,'theta_LRA']
            
#             elif dof_i == 'theta_sfootA':
#                 return m.q[1,P,'theta_sfootA'] == m.q[N,P,'theta_sfootB']
#             elif dof_i == 'theta_sfootB':
#                 return m.q[1,P,'theta_sfootB'] == m.q[N,P,'theta_sfootA']
            
            else: 
                 return Constraint.Skip
        m.def_equate_position_states_constraint = Constraint(m.DOFs, rule = def_equate_position_states) 

        
        def def_equate_velocity_states(m,dof_i):
            if dof_i == 'yb' or dof_i == 'theta_body' or dof_i == 'xb':
                return m.dq[1,0,dof_i] == m.dq[N,0,dof_i]
            
            elif dof_i == 'theta_ULA':
                return m.dq[1,0,'theta_ULA'] == m.dq[N,0,'theta_ULB']
            elif dof_i == 'theta_ULB':
                return m.dq[1,0,'theta_ULB'] == m.dq[N,0,'theta_ULA']
            
            elif dof_i == 'theta_URA':
                return m.dq[1,0,'theta_URA'] == m.dq[N,0,'theta_URB']
            elif dof_i == 'theta_URB':
                return m.dq[1,0,'theta_URB'] == m.dq[N,0,'theta_URA']
            
            elif dof_i == 'theta_LLA':
                return m.dq[1,0,'theta_LLA'] == m.dq[N,0,'theta_LLB']
            elif dof_i == 'theta_LLB':
                return m.dq[1,0,'theta_LLB'] == m.dq[N,0,'theta_LLA']
            
            elif dof_i == 'theta_LRA':
                return m.dq[1,0,'theta_LRA'] == m.dq[N,0,'theta_LRB']
            elif dof_i == 'theta_LRB':
                return m.dq[1,0,'theta_LRB'] == m.dq[N,0,'theta_LRA']
            
#             elif dof_i == 'theta_sfootA':
#                 return m.dq[1,P,'theta_sfootA'] == m.dq[N,P,'theta_sfootB']
#             elif dof_i == 'theta_sfootB':
#                 return m.dq[1,P,'theta_sfootB'] == m.dq[N,P,'theta_sfootA']
            
            else: 
                 return Constraint.Skip
        m.def_equate_velocity_states_constraint = Constraint(m.DOFs, rule = def_equate_velocity_states) 
                 
    else: 
        pass

else: pass

In [ ]:
#test code
hide_sloution('Initial Conditions')

for n in range(1,N+1):
    for p in range(P+1):
        #m.q[n,'y_body'].setlb(0.0)
        if test == 'Periodic_Walk' or test == 'Periodic_Run':
            x_low_lim = -0.1
            x_high_lim = 2.0
            y_low_lim = -0.5
            y_high_lim = 1.5

            m.q[n,p,'theta_sfootA'].setub(sfoot_max)
            m.q[n,p,'theta_sfootA'].setlb(sfoot_min)
            m.q[n,p,'theta_sfootB'].setub(sfoot_max)
            m.q[n,p,'theta_sfootB'].setlb(sfoot_min)
            
            

        else:
            x_low_lim = -0.1
            x_high_lim = 1.0
            y_low_lim = -0.5
            y_high_lim = 2.0
            
            m.q[n,p,'theta_sfootA'].setub(0.01)
            m.q[n,p,'theta_sfootA'].setlb(-0.01)
            m.q[n,p,'theta_sfootB'].setub(0.01)
            m.q[n,p,'theta_sfootB'].setlb(-0.01)


            for dof in DOFs:
                m.dq[1,0,dof].fix(0.0)
                
            if 'Drop' in test:
                [m.tau_torque[n,side,leg].fix(0.0) for side in sides for leg in legs] 
                
            

#             if 'Drop' in test:
#                 [m.tau_torque[n,p,side,leg].fix(0.0) for side in sides for leg in legs]       
#             else:
#                 # test == 'Start_Periodic_Walk' or 'Max_Hop'
#                 [m.tau_torque[1,P,side,leg].fix(0.0) for side in sides for leg in legs]

In [ ]:
# Initialize Points
hide_sloution('Seeds')

# what values to use?
# q, dq,
# forces (GRF) and torques?

for n in range(1,N+1):
    for p in range(P+1):
            #q            
            m.q[n,p,'xb'].value = np.random.uniform(0,TT*h_max*V_avg)
            m.q[n,p,'yb'].value = np.random.uniform(0.25,0.4)

            m.q[n,p,'theta_ULA'].value = np.random.uniform(-0.5*np.pi,1.5*np.pi)
            m.q[n,p,'theta_URA'].value = np.random.uniform(-0.5*np.pi,1.5*np.pi)
            m.q[n,p,'theta_ULB'].value = np.random.uniform(-0.5*np.pi,1.5*np.pi)
            m.q[n,p,'theta_URB'].value = np.random.uniform(-0.5*np.pi,1.5*np.pi)
            
            

            m.knee_angle[n,p,'left','A'].value = np.random.uniform(knee_angle_min*(np.pi)/180,knee_angle_max*(np.pi)/180)
            m.knee_angle[n,p,'right','A'].value = np.random.uniform(knee_angle_min*(np.pi)/180,knee_angle_max*(np.pi)/180)
            m.knee_angle[n,p,'left','B'].value = np.random.uniform(knee_angle_min*(np.pi)/180,knee_angle_max*(np.pi)/180)
            m.knee_angle[n,p,'right','B'].value = np.random.uniform(knee_angle_min*(np.pi)/180,knee_angle_max*(np.pi)/180)

            m.q[n,p,'theta_sfootA'].value = np.random.uniform(sfoot_min,sfoot_max)
            m.q[n,p,'theta_sfootB'].value = np.random.uniform(sfoot_min,sfoot_max)
            
            m.dq[n,p,'xb'].value = np.random.uniform(round(V_avg*0.5),round(V_avg*1.5))
            
            
            print('--------------------------------------------------------------------------------')
            print('xb= ',m.q[n,p,'xb'].value)
            print('yb= ',m.q[n,p,'yb'].value)

            print('theta_ULA= ',m.q[n,p,'theta_ULA'].value)
            print('theta_URA= ',m.q[n,p,'theta_URA'].value)
            print('theta_ULB= ',m.q[n,p,'theta_ULB'].value)
            print('theta_URB= ',m.q[n,p,'theta_URB'].value)
            
            

            print('knee_leftA= ',m.knee_angle[n,p,'left','A'].value)
            print('knee_rightA= ',m.knee_angle[n,p,'right','A'].value)
            print('knee_leftB= ',m.knee_angle[n,p,'left','B'].value)
            print('knee_rightB= ',m.knee_angle[n,p,'right','B'].value)

            print('theta_sfootA = ',m.q[n,p,'theta_sfootA'].value)
            print('theta_sfootB = ',m.q[n,p,'theta_sfootB'].value)
            
            print('dxb= ',m.dq[n,p,'xb'].value)
            print('--------------------------------------------------------------------------------')

#             for gc in ground_constraints:
#                 m.ground_penalty[n,gc,leg].value = np.random.uniform(0.0,1e-3)




#             for leg in legs:
#                 for side in sides:
#                     m.tau_torque[n,p,side,leg].value = np.random.uniform(-desired_max_torque,desired_max_torque)
                    
#                     for sign in signs:
#                         m.motor_velocity[n,p,side,leg,sign].value = np.random.uniform(0.0,no_load_speed)
                        
#                 for sign in signs:
#                     m.GRF_x[n,p,sign,leg].value = np.random.uniform(0.0,lim_GRF)
                    
#                 m.GRF_y[n,p,leg].value = np.random.uniform((m.GRF_x[n,p,'+ve',leg].value + m.GRF_x[n,p,'-ve',leg].value)/m.µ.value)   
                
#                 for wdof in WDOFs:
#                     m.connect_force[n,p,wdof,leg].value = np.random.uniform(-10*total_mass*9.81,10*total_mass*9.81)
             
            
            
            #m.dq[n,p,'yb'].value = np.random.uniform(-0.3,0.4)
            # dq --> velocity of knee angle                    
#             m.dq[n,p,'sfootA'].value = np.random.uniform(sfoot_min,sfoot_max)
#             m.dq[n,p,'sfootB'].value = np.random.uniform(sfoot_min,sfoot_max)
            
            # velocity for knee angle?
#             m.knee_angle[n,p,'left','A'].value = np.random.uniform(knee_angle_min,knee_angle_max)
#             m.knee_angle[n,p,'right','A'].value = np.random.uniform(knee_angle_min,knee_angle_max)
#             m.knee_angle[n,p,'left','B'].value = np.random.uniform(knee_angle_min,knee_angle_max)
#             m.knee_angle[n,p,'right','B'].value = np.random.uniform(knee_angle_min,knee_angle_max)
       

In [ ]:
hide_sloution('Transition Torque')
def def_match_tau(m,n,side,leg):
        
        if n == 1:
            if side == 'left':
                if leg == 'A':             
                    return m.tau_torque[n,'left','A'] == m.tau_torque[N,'left','B'] 
                else:
                    return m.tau_torque[n,'left','B'] == m.tau_torque[N,'left','A']
            
            else:
                if leg == 'A':             
                    return m.tau_torque[n,'right','A'] == m.tau_torque[N,'right','B'] 
                else:
                    return m.tau_torque[n,'right','B'] == m.tau_torque[N,'right','A']
            
            
        elif n == 2:
            if side == 'left':
                if leg == 'A':             
                    return m.tau_torque[n,'left','A'] == m.tau_torque[N-1,'left','B'] 
                else:
                    return m.tau_torque[n,'left','B'] == m.tau_torque[N-1,'left','A']
                
            else:
                if leg == 'A':             
                    return m.tau_torque[n,'right','A'] == m.tau_torque[N-1,'right','B'] 
                else:
                    return m.tau_torque[n,'right','B'] == m.tau_torque[N-1,'right','A']
                
        elif n == 3:
            if side == 'left':
                if leg == 'A':             
                    return m.tau_torque[n,'left','A'] == m.tau_torque[N-2,'left','B'] 
                else:
                    return m.tau_torque[n,'left','B'] == m.tau_torque[N-2,'left','A']
            else:
                if leg == 'A':             
                    return m.tau_torque[n,'right','A'] == m.tau_torque[N-2,'right','B'] 
                else:
                    return m.tau_torque[n,'right','B'] == m.tau_torque[N-2,'right','A']  
                
        elif n == 4:
            if side == 'left':
                if leg == 'A':             
                    return m.tau_torque[n,'left','A'] == m.tau_torque[N-3,'left','B'] 
                else:
                    return m.tau_torque[n,'left','B'] == m.tau_torque[N-3,'left','A']
            else:
                if leg == 'A':             
                    return m.tau_torque[n,'right','A'] == m.tau_torque[N-3,'right','B'] 
                else:
                    return m.tau_torque[n,'right','B'] == m.tau_torque[N-3,'right','A']  
                
        elif n == 5:
            if side == 'left':
                if leg == 'A':             
                    return m.tau_torque[n,'left','A'] == m.tau_torque[N-4,'left','B'] 
                else:
                    return m.tau_torque[n,'left','B'] == m.tau_torque[N-4,'left','A']
            else:
                if leg == 'A':             
                    return m.tau_torque[n,'right','A'] == m.tau_torque[N-4,'right','B'] 
                else:
                    return m.tau_torque[n,'right','B'] == m.tau_torque[N-4,'right','A']  
        
        else: 
            return Constraint.Skip
m.def_match_tau_constraint = Constraint(m.N,m.sides,m.legs, rule = def_match_tau)
    

In [ ]:
# Hop_startup

if test == 'High_Drop':
    m.q[1,0,'xb'].fix(0.0) 
    m.q[1,0,'yb'].fix(0.5) 
    m.foot_position[1,0,'X','A'].fix(0)
    m.foot_position[1,0,'X','B'].fix(0)
    m.foot_position[1,0,'Y','A'].fix(0.1)
    m.foot_position[1,0,'Y','B'].fix(0.1)
    m.q[1,0,'theta_sfootA'].fix(0.0)
    m.q[1,0,'theta_sfootB'].fix(0.0)
    
                       
        
    
elif test == 'Periodic_Walk':
    m.q[1,0,'xb'].fix(0.0)

    m.q[1,0,'theta_sfootB'].fix(0.0)
    m.dq[1,0,'theta_sfootB'].fix(0.0)
    m.foot_position[1,0,'Y','B'].setlb(0.05)
    
    if V_avg == 0.5:
        # walk
        for n in range(1,N+1):
            for p in range(P+1):
                
                if n < round(N*0.90):
                        m.foot_position[n,p,'Y','A'].fix(0.0)
                        m.foot_velocity_x[n,p,'A'].fix(0.0)     
                else:
                    m.GRF_y[n,p,'A'].fix(0.0)

                    
                if n < round(N*0.65):
                    m.GRF_y[n,p,'B'].fix(0.0)
                else:
                    m.foot_position[n,p,'Y','B'].fix(0.0)
                    m.foot_velocity_x[n,p,'B'].fix(0.0)
                    
    if V_avg == 1.0:
        # fast walk
        for n in range(1,N+1):
            for p in range(P+1):
                if n < round(N*0.75):
                    m.foot_position[n,p,'Y','A'].fix(0.0)
                    m.foot_velocity_x[n,p,'A'].fix(0.0)     
                else:
                    m.GRF_y[n,p,'A'].fix(0.0)
                    
                if n < round(N*0.35):
                    m.GRF_y[n,p,'B'].fix(0.0)
                else:
                    m.foot_position[n,p,'Y','B'].fix(0.0)
                    m.foot_velocity_x[n,p,'B'].fix(0.0)
    else: pass

elif test == 'Periodic_Run':
    m.q[1,0,'xb'].fix(0.0)    
    
    if V_avg == 1.5:
        # jog
        m.q[1,0,'theta_sfootB'].fix(0.0)
        m.dq[1,0,'theta_sfootB'].fix(0.0)
        m.foot_position[1,0,'Y','B'].setlb(0.03)
        
        for n in range(1,N+1):
            for p in range(P+1): 
#                 m.q[n,p,'yb'].setlb(0.27)

                if n<round(0.75*N):
                    m.foot_position[n,p,'Y','A'].fix(0.0)
                    m.foot_velocity_x[n,p,'A'].fix(0.0) 
                else:
                    m.GRF_y[n,p,'A'].fix(0.0)

                if n<round(0.9*N):
                    m.GRF_y[n,p,'B'].fix(0.0)
                    
                else:
                    m.foot_position[n,p,'Y','B'].fix(0.0)
                    m.foot_velocity_x[n,p,'B'].fix(0.0) 
        
        
        # jog
#         m.q[1,0,'theta_sfootB'].fix(0.0)
#         m.dq[1,0,'theta_sfootB'].fix(0.0)
#         m.foot_position[1,0,'Y','B'].setlb(0.03)
        
#         for n in range(1,N+1):
#             for p in range(P+1): 
# #                 m.q[n,p,'yb'].setlb(0.27)

#                 if n<round(0.75*N):
#                     m.foot_position[n,p,'Y','A'].fix(0.0)
#                     m.foot_velocity_x[n,p,'A'].fix(0.0) 
#                 else:
#                     m.GRF_y[n,p,'A'].fix(0.0)

#                 if n<round(0.9*N):
#                     m.GRF_y[n,p,'B'].fix(0.0)
                    
#                 else:
#                     m.foot_position[n,p,'Y','B'].fix(0.0)
#                     m.foot_velocity_x[n,p,'B'].fix(0.0) 
  
    
    elif V_avg > 1.5:
    
        m.q[1,0,'theta_sfootA'].fix(0.0)
        m.dq[1,0,'theta_sfootA'].fix(0.0)


        for n in range(1,N+1):
            for p in range(P+1):
                m.q[n,p,'yb'].setlb(0.23)
#                 m.GRF_y[n,p,'B'].fix(0.0)
                
                if V_avg == 2.0:
                    m.GRF_y[n,p,'B'].fix(0.0)
                    if n>round(0.4*N) and n<round(0.75*N):
                        m.foot_position[n,p,'Y','A'].fix(0.0)
                        m.foot_velocity_x[n,p,'A'].fix(0.0)
                        m.foot_position[n,p,'Y','B'].setlb(0.01)
                        
                    else:
                        m.GRF_y[n,p,'A'].fix(0.0)    
                    
#                     if n>round(0.50*N) and n<round(0.80*N):
#                         m.foot_position[n,p,'Y','A'].fix(0.0)
#                         m.foot_velocity_x[n,p,'A'].fix(0.0)
#                         m.foot_position[n,p,'Y','B'].setlb(0.03)
                        
#                     else:
#                         m.GRF_y[n,p,'A'].fix(0.0)

                elif V_avg == 3.0:
                    m.GRF_y[n,p,'B'].fix(0.0)
                    if n>round(NC_lower*N) and n<round(NC_upper*N):
                        m.foot_position[n,p,'Y','A'].fix(0.0)
                        m.foot_velocity_x[n,p,'A'].fix(0.0)
                        m.foot_position[n,p,'Y','B'].setlb(0.01)
                        
                    else:
                        m.GRF_y[n,p,'A'].fix(0.0)       
                        
                else: pass
                

In [ ]:
# COST FUNCTION -------------------------------------------------------------------------------------------------------------
hide_sloution('Cost Function')
# m.del_component(CostFun)
cost_function_str = ''

def CostFun1(m):
    row = 1e3 #row = 1e3 # scale the penalty
    tau_sum = 0.0
    pen_sum = 0.0
    transport_sum = 0.0
    T = sum(m.h[n] for n in range(1,N+1))
    
    for n in range(1,N+1):
        for leg in legs:
            for side in sides:
                tau_sum += (hm*m.h[n]*(m.tau_torque[n,side,leg])**2)/(total_mass*9.81)
                #transport_sum+= (hm*m.h[n]*(m.tau_torque[n,side,leg])**2)/(TT*V_avg*total_mass*9.81)

            for gc in ground_constraints:
                    pen_sum += m.ground_penalty[n,gc,leg] 
        
        global cost_function_str
#         cost_function_str = 'row*pen_sum + tau_sum'
        cost_function_str = 'pen_sum'
        cost_function_return = eval(cost_function_str) #+ tau_diff/100
    return  cost_function_return
m.Cost1 = Objective(rule = CostFun1)


def CostFun2(m):
    row = 1e3 #row = 1e3 # scale the penalty
    tau_sum = 0.0
    pen_sum = 0.0
    transport_sum = 0.0
    
    T = sum(m.h[n] for n in range(1,N+1))
    
    for n in range(1,N+1):
        for leg in legs:
            for side in sides:
                tau_sum += (hm*m.h[n]*(m.tau_torque[n,side,leg])**2)/(total_mass*9.81)
                #transport_sum+= (hm*m.h[n]*(m.tau_torque[n,side,leg])**2)/(TT*V_avg*total_mass*9.81)

            for gc in ground_constraints:
                    pen_sum += m.ground_penalty[n,gc,leg] 
        
        global cost_function_str
        cost_function_str = 'row*pen_sum + tau_sum'
#         cost_function_str = 'pen_sum'
        cost_function_return = eval(cost_function_str) #+ tau_diff/100
    return  cost_function_return
m.Cost2 = Objective(rule = CostFun2)
 


m.Cost2.deactivate()

In [ ]:
def def_EOMs(m,n,p,dof_i):
    if p == 0: return Constraint.Skip
    var_list = get_var_list_values(m,n,p)
    return lamb_EOMs[dof_i](*var_list) == 0
m.def_EOMs_constraint = Constraint(m.N, m.P,m.DOFs, rule = def_EOMs)

In [ ]:
# solving
#opt = SolverFactory('ipopt',executable = '/home/uct/Trajectory_Optimization/IPOPT/build/bin/ipopt')
bambi = '/home/uct/Trajectory_Optimization/IPOPT/build/bin/ipopt'
lenovo = '/home/zubair/build/bin/ipopt'
godzilla = '/usr/local/bin/ipopt'
opt = SolverFactory('ipopt', executable = godzilla)
opt.options["linear_solver"] = 'ma86'

# solver options
import time
start_timer = time.time()

opt.options["expect_infeasible_problem"] = 'yes'
#pt.options["linear_system_scaling"] = 'none'
#opt.options["mu_strategy"] = "adaptive"
opt.options["print_level"] = 5     # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000*20   # maximum number of iterations
opt.options["max_cpu_time"] = 1000*20 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6          # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
# opt.options["OF_acceptable_obj_change_tol"] = 1e-4


results = opt.solve(m, tee = True)

In [ ]:
# PLOT COST FUNCTION -------------------------------------------------------------------------------------------------------------
# T = sum(m.h[n] for n in range(1,N+1))
hide_sloution('Check')
end_timer = time.time()

print("The time of execution of above program is :",((end_timer-start_timer)/60)/60, "hrs")

print('result = ',results.solver.termination_condition) # check if optimal
pen_sum = 0.0
tau_sum = 0.0
for n in range(1,N+1):
        for gc in ground_constraints:
            for leg in legs:
                pen_sum += m.ground_penalty[n,gc,leg].value
        for leg in legs:
            for side in sides:
                tt = m.tau_torque[n,side,leg].value
                if m.tau_torque[n,side,leg].value == None:
                    tt= 0
                tau_sum += (hm*m.h[n].value*(tt)**2)/(total_mass*9.81) 
            
            
print('ground_penalty = ',pen_sum)
T = 0
for n in range (2,N+1):
    T = T+hm*m.h[n].value
print('sum_h = ',T)
print('TT = ',TT)
print('N = ',N)
print('cost = ',tau_sum+pen_sum)
print('tau_sum = ',tau_sum)

In [ ]:
hide_sloution('Plot Solution')

#animate it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

# arrays to store values
N_time = []

def plot_robot(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-2,x_high_lim])
    ax.set_ylim([y_low_lim,y_high_lim])
    N_time.append(i)
     
    plot_body = {}
    plot_boom = {}
    plot_boom_COM = {}
    plot_UR = {}
    plot_LR = {}
    plot_UL = {}
    plot_LL = {}
    plot_foot = {}
    abs_angles = {}
    bod_angle = 0
    
    # plot body
    plot_body.update({('x_back'):m.q[i,0,'xb'].value - 0.5*m.lengths['body']*cos(bod_angle)})
    plot_body.update({('y_back'):m.q[i,0,'yb'].value - m.yBO.value - 0.5*m.lengths['body']*sin(bod_angle)})
    plot_body.update({('x_front'):m.q[i,0,'xb'].value + 0.5*m.lengths['body']*cos(bod_angle)})
    plot_body.update({('y_front'):m.q[i,0,'yb'].value - m.yBO.value + 0.5*m.lengths['body']*sin(bod_angle)})  
    ax.plot([plot_body['x_back'],plot_body['x_front']],[plot_body['y_back'],plot_body['y_front']],color='xkcd:brown')
    
#     # plot boom
#     plot_boom.update({('x_back'): 0.0})
#     plot_boom.update({('y_back'): m.y_bodyOffset.value})
#     plot_boom.update({('x_front'):m.q[i,P,'x_body'].value })
#     plot_boom.update({('y_front'):m.q[i,P,'y_body'].value })  
#     ax.plot([plot_boom['x_back'],plot_boom['x_front']],[plot_boom['y_back'],plot_boom['y_front']],color='xkcd:black')

#     # plot boom COM
#     plot_boom_COM.update({('x'): (m.cg.value/m.lx_boom.value)*m.q[i,P,'x_body'].value})
#     plot_boom_COM.update({('y'): (m.cg.value/m.ly_boom.value)*m.q[i,P,'y_body'].value})
#     ax.plot([plot_boom_COM['x']],[plot_boom_COM['y']],color='xkcd:purple',marker="o")

    for leg in legs:
        if leg == 'A':
            link_colour = ['black','red']
        else:
            link_colour = ['orange','purple']
        
        
        #plot upper right
        #abs_angles.update({('RU',leg): bod_angle + m.q[i,P,'theta_UR'+leg].value})
        plot_UR.update({('top','X',leg): m.q[i,0,'xb'].value + m.xBO.value*cos(bod_angle)})
        plot_UR.update({('top','Y',leg): m.q[i,0,'yb'].value - m.yBO.value + m.xBO.value*sin(bod_angle)})
        plot_UR.update({('bot','X',leg): plot_UR['top','X',leg] - m.lengths['UR'+leg]*cos(m.q[i,0,'theta_UR'+leg].value)})
        plot_UR.update({('bot','Y',leg): plot_UR['top','Y',leg] - m.lengths['UR'+leg]*sin(m.q[i,0,'theta_UR'+leg].value)})
        ax.plot([plot_UR['top','X',leg],plot_UR['bot','X',leg]],[plot_UR['top','Y',leg],plot_UR['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[0]))
        
        #plot lower right
        #abs_angles.update({('RL',leg): m.q[i,P,'theta_LR'+leg].value - (np.pi-abs_angles['RU',leg])})
        plot_LR.update({('top','X',leg): plot_UR['bot','X',leg]})
        plot_LR.update({('top','Y',leg): plot_UR['bot','Y',leg]})
        plot_LR.update({('bot','X',leg): plot_UR['bot','X',leg] - m.lengths['LR'+leg]*cos(m.q[i,0,'theta_LR'+leg].value-m.angle_offset.value)})
        plot_LR.update({('bot','Y',leg): plot_UR['bot','Y',leg] - m.lengths['LR'+leg]*sin(m.q[i,0,'theta_LR'+leg].value-m.angle_offset.value)})
        ax.plot([plot_LR['top','X',leg],plot_LR['bot','X',leg]],[plot_LR['top','Y',leg],plot_LR['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[0]))
        
        #plot foot
        plot_foot.update({('top','X',leg): plot_LR['bot','X',leg]})
        plot_foot.update({('top','Y',leg): plot_LR['bot','Y',leg]})
        plot_foot.update({('bot','X',leg): plot_LR['bot','X',leg] + (m.foot_length.value+m.q[i,0,'theta_sfoot'+leg].value)*cos(m.foot_angle.value - m.q[i,0,'theta_LR'+leg].value - m.angle_offset.value)})
        plot_foot.update({('bot','Y',leg): plot_LR['bot','Y',leg] - (m.foot_length.value+m.q[i,0,'theta_sfoot'+leg].value)*sin(m.foot_angle.value - m.q[i,0,'theta_LR'+leg].value - m.angle_offset.value)})
        ax.plot([plot_foot['top','X',leg],plot_foot['bot','X',leg]],[plot_foot['top','Y',leg],plot_foot['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[1]))

        #plot upper left
        #abs_angles.update({('LU',leg): bod_angle + m.q[i,P,'theta_UL'+leg].value})
        plot_UL.update({('top','X',leg): m.q[i,0,'xb'].value - m.xBO.value*cos(bod_angle)})
        plot_UL.update({('top','Y',leg): m.q[i,0,'yb'].value - m.yBO.value - m.xBO.value*sin(bod_angle)})
        plot_UL.update({('bot','X',leg): plot_UL['top','X',leg] - m.lengths['UL'+leg]*cos(m.q[i,0,'theta_UL'+leg].value)})
        plot_UL.update({('bot','Y',leg): plot_UL['top','Y',leg] - m.lengths['UL'+leg]*sin(m.q[i,0,'theta_UL'+leg].value)})
        ax.plot([plot_UL['top','X',leg],plot_UL['bot','X',leg]],[plot_UL['top','Y',leg],plot_UL['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[0]))

        #plot lower left
        #abs_angles.update({('LL',leg):m.q[i,P,'theta_LL'+leg].value - (np.pi-abs_angles['LU',leg])})
        plot_LL.update({('top','X',leg): plot_UL['bot','X',leg]})
        plot_LL.update({('top','Y',leg): plot_UL['bot','Y',leg]})
        plot_LL.update({('bot','X',leg): plot_UL['bot','X',leg] - m.lengths['LL'+leg]*cos(m.q[i,0,'theta_LL'+leg].value)})
        plot_LL.update({('bot','Y',leg): plot_UL['bot','Y',leg] - m.lengths['LL'+leg]*sin(m.q[i,0,'theta_LL'+leg].value)})
        ax.plot([plot_LL['top','X',leg],plot_LL['bot','X',leg]],[plot_LL['top','Y',leg],plot_LL['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[0]))     
    
    
    if m.GRF_y[i,0,'A'].value > 1.0:
        ax.arrow(plot_LL['bot','X','A'], plot_LL['bot','Y','A'], 0, 0.01*m.GRF_y[i,0,'A'].value,color='xkcd:{0}'.format('red'),head_width = 0.1)
        
    if m.GRF_y[i,0,'B'].value > 1.0:
        ax.arrow(plot_LR['bot','X','B'], plot_LR['bot','Y','B'], 0, 0.01*m.GRF_y[i,0,'B'].value,color='xkcd:{0}'.format('red'),head_width = 0.1)
    
#     if m.GRF_x[i,P,'-ve','B'].value > 0.0:
#         #print('GRF_footL = ',m.GRF_y[i,P,'footL'].value)
#         ax.arrow(plot_LR['bot','X','B'], plot_LR['bot','Y','B'], 0, 0.01*m.GRF_x[i,P,'-ve','B'].value,color='xkcd:{0}'.format('blue'),head_width = 0.1)
    
    
    #ax.legend(["GRF*0.01"], frameon=True)
    ax.text(x_low_lim, y_high_lim, '0.01*GRF', color = 'red',fontsize = 15)
    
    ax.plot([-10,10],[0,0]) # plot the ground
    
    
update = lambda i: plot_robot(i,m,ax1) # lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook


In [ ]:
hide_sloution('PLOTS')
plt.figure()
saveGraphs = False
x = np.arange(1,(N)*(P)+1,1)

nodes = []
h_values = []

xbody = []
ybody = []
dxbody = []
dybody = []
ddxbody = []
ddybody = []
    
SS_thULA = []
SS_thURA = []
SS_thULB = []
SS_thURB = []


SS_dthULA = []
SS_dthURA = []
SS_dthULB = []
SS_dthURB = []

SS_Tau_LeftA = []
SS_Tau_RightA = []
SS_Tau_LeftB = []
SS_Tau_RightB = []

GRF_yA = []
GRF_yB = []
GRF_xA = []
GRF_xB = []

foot_position_Y_A = []
foot_position_Y_B = []
foot_position_X_A = []
foot_position_X_B = []

foot_velocity_Y_A = []
foot_velocity_Y_B = []
foot_velocity_X_A = []
foot_velocity_X_B = []

knee_angle_LA = []
knee_angle_LB = []
knee_angle_RA = []
knee_angle_RB = []

sfootA = []
sfootB = []

# ankle_body_Y_A = []
# ankle_body_Y_B = []

def fix_array(value):
    if value == None:
        return 0
    else:
        return value

for n in range(1,N+1):
    for p in range(0,P):
        nodes.append(n)
        h_values.append(hm*m.h[n].value)

        xbody.append(fix_array(m.q[n,p,'xb'].value))
        ybody.append(fix_array(m.q[n,p,'yb'].value))
        dxbody.append(fix_array(m.dq[n,p,'xb'].value))
        dybody.append(fix_array(m.dq[n,p,'yb'].value))
        ddxbody.append(fix_array(m.ddq[n,p,'xb'].value))
        ddybody.append(fix_array(m.ddq[n,p,'yb'].value))

        SS_thULA.append((np.pi/2)-zero_offset-fix_array(m.q[n,p,'theta_ULA'].value)) 
        SS_thURA.append(zero_offset+(np.pi/2)-fix_array(m.q[n,p,'theta_URA'].value)) 
        SS_thULB.append((np.pi/2)-zero_offset-fix_array(m.q[n,p,'theta_ULB'].value)) 
        SS_thURB.append(zero_offset+(np.pi/2)-fix_array(m.q[n,p,'theta_URB'].value)) 

        SS_dthULA.append(-fix_array(m.dq[n,p,'theta_ULA'].value)) 
        SS_dthURA.append(-fix_array(m.dq[n,p,'theta_URA'].value)) 
        SS_dthULB.append(-fix_array(m.dq[n,p,'theta_ULB'].value)) 
        SS_dthURB.append(-fix_array(m.dq[n,p,'theta_URB'].value)) 

        SS_Tau_LeftA.append(-fix_array(m.tau_torque[n,'left','A'].value))
        SS_Tau_RightA.append(-fix_array(m.tau_torque[n,'right','A'].value))
        SS_Tau_LeftB.append(-fix_array(m.tau_torque[n,'left','B'].value))
        SS_Tau_RightB.append(-fix_array(m.tau_torque[n,'right','B'].value))

        GRF_yA.append(fix_array(m.GRF_y[n,p,'A'].value))
        GRF_yB.append(fix_array(m.GRF_y[n,p,'B'].value))
        GRF_xA.append(fix_array(m.GRF_x[n,p,'+ve','A'].value) - fix_array(m.GRF_x[n,p,'-ve','A'].value))
        GRF_xB.append(fix_array(m.GRF_x[n,p,'+ve','B'].value) - fix_array(m.GRF_x[n,p,'-ve','B'].value))

        foot_position_Y_A.append(fix_array(m.foot_position[n,p,'Y','A'].value))
        foot_position_Y_B.append(fix_array(m.foot_position[n,p,'Y','B'].value))
        foot_position_X_A.append(fix_array(m.foot_position[n,p,'X','A'].value))
        foot_position_X_B.append(fix_array(m.foot_position[n,p,'X','B'].value))

        foot_velocity_X_A.append(fix_array(m.foot_velocity[n,p,'X','+ve','A'].value)-fix_array(m.foot_velocity[n,p,'X','-ve','A'].value))
        foot_velocity_X_B.append(fix_array(m.foot_velocity[n,p,'X','+ve','B'].value)-fix_array(m.foot_velocity[n,p,'X','-ve','B'].value))
        foot_velocity_Y_A.append(fix_array(m.foot_velocity[n,p,'Y','+ve','A'].value)-fix_array(m.foot_velocity[n,p,'Y','-ve','A'].value))
        foot_velocity_Y_B.append(fix_array(m.foot_velocity[n,p,'Y','+ve','B'].value)-fix_array(m.foot_velocity[n,p,'Y','-ve','B'].value))

        knee_angle_LA.append(fix_array(m.knee_angle[n,p,'left','A'].value)*180/np.pi)
        knee_angle_LB.append(fix_array(m.knee_angle[n,p,'left','B'].value)*180/np.pi)
        knee_angle_RA.append(fix_array(m.knee_angle[n,p,'right','A'].value)*180/np.pi)
        knee_angle_RB.append(fix_array(m.knee_angle[n,p,'right','B'].value)*180/np.pi)

        sfootA.append(fix_array(m.q[n,p,'theta_sfootA'].value))
        sfootB.append(fix_array(m.q[n,p,'theta_sfootB'].value))

    #     ankle_body_Y_A.append(m.ankle_body_Y[n,p,'A'].value))
    #     ankle_body_Y_B.append(m.ankle_body_Y[n,p,'B'].value))

plt.figure()
plt.plot(x,sfootA,label = 'sfootA')
plt.plot(x,sfootB,label = 'sfootB')
plt.title('sfeet')
plt.legend()

plt.figure()
plt.plot(x,SS_Tau_LeftA,label = 'LA')
plt.plot(x,SS_Tau_LeftB,label = 'LB')
plt.plot(x,SS_Tau_RightA,label = 'RA')
plt.plot(x,SS_Tau_RightB,label = 'RB')
plt.title('TorqueCurve')
plt.legend()

plt.figure()
fig, axs = plt.subplots(2,1)   
axs[0].plot(x,GRF_yA,label = 'A')
axs[0].plot(x,GRF_yB,label = 'B')
axs[1].plot(x,GRF_xA,label = 'A')
axs[1].plot(x,GRF_xB,label = 'B')
axs[0].set_title('GRF_Y')
axs[1].set_title('GRF_X')
fig.legend()

plt.figure()
fig, axs = plt.subplots(2,1)
axs[0].plot(x,foot_position_Y_A,label = 'A')
axs[0].plot(x,foot_position_Y_B,label = 'B')
axs[1].plot(x,foot_velocity_X_A,label = 'A')
axs[1].plot(x,foot_velocity_X_B,label = 'B')
axs[0].set_title('foot_position_Y')
axs[1].set_title('foot_velocity_X')
fig.legend()

plt.figure()
plt.plot(x,knee_angle_LA,label = 'LA')
plt.plot(x,knee_angle_LB,label = 'LB')
plt.plot(x,knee_angle_RA,label = 'RA')
plt.plot(x,knee_angle_RB,label = 'RB')
plt.title('Knee_Angle')
plt.legend()

plt.figure()
plt.plot(x,dxbody,label = 'dxbody')
plt.plot(x,dybody,label = 'dybody')
plt.title('body velocity')
plt.legend()

plt.figure()
plt.plot(x,ddybody,label = 'ddybody')
plt.title('acceleration')
plt.legend()


plt.figure()
plt.plot(x,SS_thULA,label = 'SS_thULA')
plt.plot(x,SS_thULB,label = 'SS_thULB')
plt.title('SS_thUL')
plt.legend()

plt.figure()
plt.plot(x,SS_thURA,label = 'SS_thURA')
plt.plot(x,SS_thURB,label = 'SS_thURB')
plt.title('SS_thUR')
plt.legend()

plt.figure()
plt.plot(x,SS_dthULA,label = 'SS_dthULA')
plt.plot(x,SS_dthULB,label = 'SS_dthULB')
plt.title('SS_dthUL')
plt.legend()

plt.figure()
plt.plot(x,SS_dthURA,label = 'SS_dthURA')
plt.plot(x,SS_dthURB,label = 'SS_dthURB')
plt.title('SS_dthUR')
plt.legend()


plt.figure()
plt.plot(x,SS_Tau_LeftA,label = 'SS_Tau_LeftA')
plt.plot(x,SS_Tau_LeftB,label = 'SS_Tau_LeftB')
plt.title('SS_TauLeft')
plt.legend()

plt.figure()
plt.plot(x,SS_Tau_RightA,label = 'SS_Tau_RightA')
plt.plot(x,SS_Tau_RightB,label = 'SS_Tau_RightB')
plt.title('SS_TauRight')
plt.legend()



plt.figure()
plt.plot(SS_dthULA,SS_Tau_LeftA)
plt.plot(SS_dthURA,SS_Tau_RightA)
plt.plot(SS_dthULB,SS_Tau_LeftB)
plt.plot(SS_dthURB,SS_Tau_RightB)
plt.grid()
plt.plot(x_p,rated_torque_p,color = 'green',linestyle = '--')
plt.plot(x_n,rated_torque_n,color = 'green',linestyle = '--')
plt.plot(x_p, y_p,'b')
plt.plot(x_n, stall_torque_p,'b')
plt.fill_between(x_p, y_p, alpha=0.4, color = 'cyan')
plt.fill_between(x_n, stall_torque_p, alpha=0.4,color = 'cyan')
plt.plot(x_n, y_n,'r')
plt.plot(x_p, stall_torque_n,'r')
plt.fill_between(x_n, y_n, alpha=0.4,color = 'pink')
plt.fill_between(x_p, stall_torque_n, alpha=0.4,color = 'pink')
plt.axis((-40,40,-92,92))
plt.title('New Torque Speed Curve')
plt.xlabel('Speed (rad/s)')
plt.ylabel('Torque (Nm)')
#plt.legend(["Tw_leftA", "Tw_rightA","Tw_leftB", "Tw_rightB","rated_torque"], loc=0, frameon=True)

In [ ]:
m.Cost1.deactivate()
m.Cost2.activate()

# solving
#opt = SolverFactory('ipopt',executable = '/home/uct/Trajectory_Optimization/IPOPT/build/bin/ipopt')
bambi = '/home/uct/Trajectory_Optimization/IPOPT/build/bin/ipopt'
lenovo = '/home/zubair/build/bin/ipopt'
godzilla = '/usr/local/bin/ipopt'
opt = SolverFactory('ipopt', executable = godzilla)
opt.options["linear_solver"] = 'ma86'

# solver options
import time
start_timer = time.time()

opt.options["expect_infeasible_problem"] = 'yes'
#pt.options["linear_system_scaling"] = 'none'
#opt.options["mu_strategy"] = "adaptive"
opt.options["print_level"] = 5     # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000*20   # maximum number of iterations
opt.options["max_cpu_time"] = 1000*20 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6          # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
# opt.options["OF_acceptable_obj_change_tol"] = 1e-4


results = opt.solve(m, tee = True)


In [ ]:
# PLOT COST FUNCTION -------------------------------------------------------------------------------------------------------------
# T = sum(m.h[n] for n in range(1,N+1))
hide_sloution('Check')
end_timer = time.time()

print("The time of execution of above program is :",((end_timer-start_timer)/60)/60, "hrs")

print('result = ',results.solver.termination_condition) # check if optimal
pen_sum = 0.0
tau_sum = 0.0
for n in range(1,N+1):
        for gc in ground_constraints:
            for leg in legs:
                pen_sum += m.ground_penalty[n,gc,leg].value
        for leg in legs:
            for side in sides:
                tt = m.tau_torque[n,side,leg].value
                if m.tau_torque[n,side,leg].value == None:
                    tt= 0
                tau_sum += (hm*m.h[n].value*(tt)**2)/(total_mass*9.81) 
            
            
print('ground_penalty = ',pen_sum)
T = 0
for n in range (2,N+1):
    T = T+hm*m.h[n].value
print('sum_h = ',T)
print('TT = ',TT)
print('N = ',N)
print('cost = ',tau_sum+pen_sum)
print('tau_sum = ',tau_sum)

In [ ]:
hide_sloution('Plot Solution')

#animate it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

# arrays to store values
N_time = []

def plot_robot(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-2,x_high_lim])
    ax.set_ylim([y_low_lim,y_high_lim])
    N_time.append(i)
     
    plot_body = {}
    plot_boom = {}
    plot_boom_COM = {}
    plot_UR = {}
    plot_LR = {}
    plot_UL = {}
    plot_LL = {}
    plot_foot = {}
    abs_angles = {}
    bod_angle = 0
    
    # plot body
    plot_body.update({('x_back'):m.q[i,0,'xb'].value - 0.5*m.lengths['body']*cos(bod_angle)})
    plot_body.update({('y_back'):m.q[i,0,'yb'].value - m.yBO.value - 0.5*m.lengths['body']*sin(bod_angle)})
    plot_body.update({('x_front'):m.q[i,0,'xb'].value + 0.5*m.lengths['body']*cos(bod_angle)})
    plot_body.update({('y_front'):m.q[i,0,'yb'].value - m.yBO.value + 0.5*m.lengths['body']*sin(bod_angle)})  
    ax.plot([plot_body['x_back'],plot_body['x_front']],[plot_body['y_back'],plot_body['y_front']],color='xkcd:brown')
    
#     # plot boom
#     plot_boom.update({('x_back'): 0.0})
#     plot_boom.update({('y_back'): m.y_bodyOffset.value})
#     plot_boom.update({('x_front'):m.q[i,P,'x_body'].value })
#     plot_boom.update({('y_front'):m.q[i,P,'y_body'].value })  
#     ax.plot([plot_boom['x_back'],plot_boom['x_front']],[plot_boom['y_back'],plot_boom['y_front']],color='xkcd:black')

#     # plot boom COM
#     plot_boom_COM.update({('x'): (m.cg.value/m.lx_boom.value)*m.q[i,P,'x_body'].value})
#     plot_boom_COM.update({('y'): (m.cg.value/m.ly_boom.value)*m.q[i,P,'y_body'].value})
#     ax.plot([plot_boom_COM['x']],[plot_boom_COM['y']],color='xkcd:purple',marker="o")

    for leg in legs:
        if leg == 'A':
            link_colour = ['black','red']
        else:
            link_colour = ['orange','purple']
        
        
        #plot upper right
        #abs_angles.update({('RU',leg): bod_angle + m.q[i,P,'theta_UR'+leg].value})
        plot_UR.update({('top','X',leg): m.q[i,0,'xb'].value + m.xBO.value*cos(bod_angle)})
        plot_UR.update({('top','Y',leg): m.q[i,0,'yb'].value - m.yBO.value + m.xBO.value*sin(bod_angle)})
        plot_UR.update({('bot','X',leg): plot_UR['top','X',leg] - m.lengths['UR'+leg]*cos(m.q[i,0,'theta_UR'+leg].value)})
        plot_UR.update({('bot','Y',leg): plot_UR['top','Y',leg] - m.lengths['UR'+leg]*sin(m.q[i,0,'theta_UR'+leg].value)})
        ax.plot([plot_UR['top','X',leg],plot_UR['bot','X',leg]],[plot_UR['top','Y',leg],plot_UR['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[0]))
        
        #plot lower right
        #abs_angles.update({('RL',leg): m.q[i,P,'theta_LR'+leg].value - (np.pi-abs_angles['RU',leg])})
        plot_LR.update({('top','X',leg): plot_UR['bot','X',leg]})
        plot_LR.update({('top','Y',leg): plot_UR['bot','Y',leg]})
        plot_LR.update({('bot','X',leg): plot_UR['bot','X',leg] - m.lengths['LR'+leg]*cos(m.q[i,0,'theta_LR'+leg].value-m.angle_offset.value)})
        plot_LR.update({('bot','Y',leg): plot_UR['bot','Y',leg] - m.lengths['LR'+leg]*sin(m.q[i,0,'theta_LR'+leg].value-m.angle_offset.value)})
        ax.plot([plot_LR['top','X',leg],plot_LR['bot','X',leg]],[plot_LR['top','Y',leg],plot_LR['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[0]))
        
        #plot foot
        plot_foot.update({('top','X',leg): plot_LR['bot','X',leg]})
        plot_foot.update({('top','Y',leg): plot_LR['bot','Y',leg]})
        plot_foot.update({('bot','X',leg): plot_LR['bot','X',leg] + (m.foot_length.value+m.q[i,0,'theta_sfoot'+leg].value)*cos(m.foot_angle.value - m.q[i,0,'theta_LR'+leg].value - m.angle_offset.value)})
        plot_foot.update({('bot','Y',leg): plot_LR['bot','Y',leg] - (m.foot_length.value+m.q[i,0,'theta_sfoot'+leg].value)*sin(m.foot_angle.value - m.q[i,0,'theta_LR'+leg].value - m.angle_offset.value)})
        ax.plot([plot_foot['top','X',leg],plot_foot['bot','X',leg]],[plot_foot['top','Y',leg],plot_foot['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[1]))

        #plot upper left
        #abs_angles.update({('LU',leg): bod_angle + m.q[i,P,'theta_UL'+leg].value})
        plot_UL.update({('top','X',leg): m.q[i,0,'xb'].value - m.xBO.value*cos(bod_angle)})
        plot_UL.update({('top','Y',leg): m.q[i,0,'yb'].value - m.yBO.value - m.xBO.value*sin(bod_angle)})
        plot_UL.update({('bot','X',leg): plot_UL['top','X',leg] - m.lengths['UL'+leg]*cos(m.q[i,0,'theta_UL'+leg].value)})
        plot_UL.update({('bot','Y',leg): plot_UL['top','Y',leg] - m.lengths['UL'+leg]*sin(m.q[i,0,'theta_UL'+leg].value)})
        ax.plot([plot_UL['top','X',leg],plot_UL['bot','X',leg]],[plot_UL['top','Y',leg],plot_UL['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[0]))

        #plot lower left
        #abs_angles.update({('LL',leg):m.q[i,P,'theta_LL'+leg].value - (np.pi-abs_angles['LU',leg])})
        plot_LL.update({('top','X',leg): plot_UL['bot','X',leg]})
        plot_LL.update({('top','Y',leg): plot_UL['bot','Y',leg]})
        plot_LL.update({('bot','X',leg): plot_UL['bot','X',leg] - m.lengths['LL'+leg]*cos(m.q[i,0,'theta_LL'+leg].value)})
        plot_LL.update({('bot','Y',leg): plot_UL['bot','Y',leg] - m.lengths['LL'+leg]*sin(m.q[i,0,'theta_LL'+leg].value)})
        ax.plot([plot_LL['top','X',leg],plot_LL['bot','X',leg]],[plot_LL['top','Y',leg],plot_LL['bot','Y',leg]],color='xkcd:{0}'.format(link_colour[0]))     
    
    
    if m.GRF_y[i,0,'A'].value > 1.0:
        ax.arrow(plot_LL['bot','X','A'], plot_LL['bot','Y','A'], 0, 0.01*m.GRF_y[i,0,'A'].value,color='xkcd:{0}'.format('red'),head_width = 0.1)
        
    if m.GRF_y[i,0,'B'].value > 1.0:
        ax.arrow(plot_LR['bot','X','B'], plot_LR['bot','Y','B'], 0, 0.01*m.GRF_y[i,0,'B'].value,color='xkcd:{0}'.format('red'),head_width = 0.1)
    
#     if m.GRF_x[i,P,'-ve','B'].value > 0.0:
#         #print('GRF_footL = ',m.GRF_y[i,P,'footL'].value)
#         ax.arrow(plot_LR['bot','X','B'], plot_LR['bot','Y','B'], 0, 0.01*m.GRF_x[i,P,'-ve','B'].value,color='xkcd:{0}'.format('blue'),head_width = 0.1)
    
    
    #ax.legend(["GRF*0.01"], frameon=True)
    ax.text(x_low_lim, y_high_lim, '0.01*GRF', color = 'red',fontsize = 15)
    
    ax.plot([-10,10],[0,0]) # plot the ground
    
    
update = lambda i: plot_robot(i,m,ax1) # lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook


In [ ]:
hide_sloution('PLOTS')
plt.figure()
saveGraphs = False
x = np.arange(1,(N)*(P)+1,1)

nodes = []
h_values = []

xbody = []
ybody = []
dxbody = []
dybody = []
ddxbody = []
ddybody = []
    
SS_thULA = []
SS_thURA = []
SS_thULB = []
SS_thURB = []


SS_dthULA = []
SS_dthURA = []
SS_dthULB = []
SS_dthURB = []

SS_Tau_LeftA = []
SS_Tau_RightA = []
SS_Tau_LeftB = []
SS_Tau_RightB = []

GRF_yA = []
GRF_yB = []
GRF_xA = []
GRF_xB = []

foot_position_Y_A = []
foot_position_Y_B = []
foot_position_X_A = []
foot_position_X_B = []

foot_velocity_Y_A = []
foot_velocity_Y_B = []
foot_velocity_X_A = []
foot_velocity_X_B = []

knee_angle_LA = []
knee_angle_LB = []
knee_angle_RA = []
knee_angle_RB = []

sfootA = []
sfootB = []

# ankle_body_Y_A = []
# ankle_body_Y_B = []

def fix_array(value):
    if value == None:
        return 0
    else:
        return value

for n in range(1,N+1):
    for p in range(0,P):
        nodes.append(n)
        h_values.append(hm*m.h[n].value)

        xbody.append(fix_array(m.q[n,p,'xb'].value))
        ybody.append(fix_array(m.q[n,p,'yb'].value))
        dxbody.append(fix_array(m.dq[n,p,'xb'].value))
        dybody.append(fix_array(m.dq[n,p,'yb'].value))
        ddxbody.append(fix_array(m.ddq[n,p,'xb'].value))
        ddybody.append(fix_array(m.ddq[n,p,'yb'].value))

        SS_thULA.append((np.pi/2)-zero_offset-fix_array(m.q[n,p,'theta_ULA'].value)) 
        SS_thURA.append(zero_offset+(np.pi/2)-fix_array(m.q[n,p,'theta_URA'].value)) 
        SS_thULB.append((np.pi/2)-zero_offset-fix_array(m.q[n,p,'theta_ULB'].value)) 
        SS_thURB.append(zero_offset+(np.pi/2)-fix_array(m.q[n,p,'theta_URB'].value)) 

        SS_dthULA.append(-fix_array(m.dq[n,p,'theta_ULA'].value)) 
        SS_dthURA.append(-fix_array(m.dq[n,p,'theta_URA'].value)) 
        SS_dthULB.append(-fix_array(m.dq[n,p,'theta_ULB'].value)) 
        SS_dthURB.append(-fix_array(m.dq[n,p,'theta_URB'].value)) 

        SS_Tau_LeftA.append(-fix_array(m.tau_torque[n,'left','A'].value))
        SS_Tau_RightA.append(-fix_array(m.tau_torque[n,'right','A'].value))
        SS_Tau_LeftB.append(-fix_array(m.tau_torque[n,'left','B'].value))
        SS_Tau_RightB.append(-fix_array(m.tau_torque[n,'right','B'].value))

        GRF_yA.append(fix_array(m.GRF_y[n,p,'A'].value))
        GRF_yB.append(fix_array(m.GRF_y[n,p,'B'].value))
        GRF_xA.append(fix_array(m.GRF_x[n,p,'+ve','A'].value) - fix_array(m.GRF_x[n,p,'-ve','A'].value))
        GRF_xB.append(fix_array(m.GRF_x[n,p,'+ve','B'].value) - fix_array(m.GRF_x[n,p,'-ve','B'].value))

        foot_position_Y_A.append(fix_array(m.foot_position[n,p,'Y','A'].value))
        foot_position_Y_B.append(fix_array(m.foot_position[n,p,'Y','B'].value))
        foot_position_X_A.append(fix_array(m.foot_position[n,p,'X','A'].value))
        foot_position_X_B.append(fix_array(m.foot_position[n,p,'X','B'].value))

        foot_velocity_X_A.append(fix_array(m.foot_velocity[n,p,'X','+ve','A'].value)-fix_array(m.foot_velocity[n,p,'X','-ve','A'].value))
        foot_velocity_X_B.append(fix_array(m.foot_velocity[n,p,'X','+ve','B'].value)-fix_array(m.foot_velocity[n,p,'X','-ve','B'].value))
        foot_velocity_Y_A.append(fix_array(m.foot_velocity[n,p,'Y','+ve','A'].value)-fix_array(m.foot_velocity[n,p,'Y','-ve','A'].value))
        foot_velocity_Y_B.append(fix_array(m.foot_velocity[n,p,'Y','+ve','B'].value)-fix_array(m.foot_velocity[n,p,'Y','-ve','B'].value))

        knee_angle_LA.append(fix_array(m.knee_angle[n,p,'left','A'].value)*180/np.pi)
        knee_angle_LB.append(fix_array(m.knee_angle[n,p,'left','B'].value)*180/np.pi)
        knee_angle_RA.append(fix_array(m.knee_angle[n,p,'right','A'].value)*180/np.pi)
        knee_angle_RB.append(fix_array(m.knee_angle[n,p,'right','B'].value)*180/np.pi)

        sfootA.append(fix_array(m.q[n,p,'theta_sfootA'].value))
        sfootB.append(fix_array(m.q[n,p,'theta_sfootB'].value))

    #     ankle_body_Y_A.append(m.ankle_body_Y[n,p,'A'].value))
    #     ankle_body_Y_B.append(m.ankle_body_Y[n,p,'B'].value))

plt.figure()
plt.plot(x,sfootA,label = 'sfootA')
plt.plot(x,sfootB,label = 'sfootB')
plt.title('sfeet')
plt.legend()

plt.figure()
plt.plot(x,SS_Tau_LeftA,label = 'LA')
plt.plot(x,SS_Tau_LeftB,label = 'LB')
plt.plot(x,SS_Tau_RightA,label = 'RA')
plt.plot(x,SS_Tau_RightB,label = 'RB')
plt.title('TorqueCurve')
plt.legend()

plt.figure()
fig, axs = plt.subplots(2,1)   
axs[0].plot(x,GRF_yA,label = 'A')
axs[0].plot(x,GRF_yB,label = 'B')
axs[1].plot(x,GRF_xA,label = 'A')
axs[1].plot(x,GRF_xB,label = 'B')
axs[0].set_title('GRF_Y')
axs[1].set_title('GRF_X')
fig.legend()

plt.figure()
fig, axs = plt.subplots(2,1)
axs[0].plot(x,foot_position_Y_A,label = 'A')
axs[0].plot(x,foot_position_Y_B,label = 'B')
axs[1].plot(x,foot_velocity_X_A,label = 'A')
axs[1].plot(x,foot_velocity_X_B,label = 'B')
axs[0].set_title('foot_position_Y')
axs[1].set_title('foot_velocity_X')
fig.legend()

plt.figure()
plt.plot(x,knee_angle_LA,label = 'LA')
plt.plot(x,knee_angle_LB,label = 'LB')
plt.plot(x,knee_angle_RA,label = 'RA')
plt.plot(x,knee_angle_RB,label = 'RB')
plt.title('Knee_Angle')
plt.legend()

plt.figure()
plt.plot(x,dxbody,label = 'dxbody')
plt.plot(x,dybody,label = 'dybody')
plt.title('body velocity')
plt.legend()

plt.figure()
plt.plot(x,ddybody,label = 'ddybody')
plt.title('acceleration')
plt.legend()


plt.figure()
plt.plot(x,SS_thULA,label = 'SS_thULA')
plt.plot(x,SS_thULB,label = 'SS_thULB')
plt.title('SS_thUL')
plt.legend()

plt.figure()
plt.plot(x,SS_thURA,label = 'SS_thURA')
plt.plot(x,SS_thURB,label = 'SS_thURB')
plt.title('SS_thUR')
plt.legend()

plt.figure()
plt.plot(x,SS_dthULA,label = 'SS_dthULA')
plt.plot(x,SS_dthULB,label = 'SS_dthULB')
plt.title('SS_dthUL')
plt.legend()

plt.figure()
plt.plot(x,SS_dthURA,label = 'SS_dthURA')
plt.plot(x,SS_dthURB,label = 'SS_dthURB')
plt.title('SS_dthUR')
plt.legend()


plt.figure()
plt.plot(x,SS_Tau_LeftA,label = 'SS_Tau_LeftA')
plt.plot(x,SS_Tau_LeftB,label = 'SS_Tau_LeftB')
plt.title('SS_TauLeft')
plt.legend()

plt.figure()
plt.plot(x,SS_Tau_RightA,label = 'SS_Tau_RightA')
plt.plot(x,SS_Tau_RightB,label = 'SS_Tau_RightB')
plt.title('SS_TauRight')
plt.legend()



plt.figure()
plt.plot(SS_dthULA,SS_Tau_LeftA)
plt.plot(SS_dthURA,SS_Tau_RightA)
plt.plot(SS_dthULB,SS_Tau_LeftB)
plt.plot(SS_dthURB,SS_Tau_RightB)
plt.grid()
plt.plot(x_p,rated_torque_p,color = 'green',linestyle = '--')
plt.plot(x_n,rated_torque_n,color = 'green',linestyle = '--')
plt.plot(x_p, y_p,'b')
plt.plot(x_n, stall_torque_p,'b')
plt.fill_between(x_p, y_p, alpha=0.4, color = 'cyan')
plt.fill_between(x_n, stall_torque_p, alpha=0.4,color = 'cyan')
plt.plot(x_n, y_n,'r')
plt.plot(x_p, stall_torque_n,'r')
plt.fill_between(x_n, y_n, alpha=0.4,color = 'pink')
plt.fill_between(x_p, stall_torque_n, alpha=0.4,color = 'pink')
plt.axis((-40,40,-92,92))
plt.title('New Torque Speed Curve')
plt.xlabel('Speed (rad/s)')
plt.ylabel('Torque (Nm)')
#plt.legend(["Tw_leftA", "Tw_rightA","Tw_leftB", "Tw_rightB","rated_torque"], loc=0, frameon=True)

In [ ]:
# Python program to explain os.mkdir() method
# importing os module
saveFile =  True
# if results.solver.termination_condition == 'optimal':
#     saveFile = True
# else:
#     saveFile = False

if saveFile == True:
    save_folder = main_name+"/"
    saveGraphs = True
    import os
    path_pc = os.getcwd()+"/"
    # Path
    path = os.path.join(path_pc, save_folder)
    os.mkdir(path)
else:
    pass

In [ ]:
print('The below save files can only be run for thB =0 as the velocities will not be the same for thB !=0')
if saveFile == True:

    x = np.arange(1,N*P+1,1)  
    
    
    plt.plot(x,SS_Tau_LeftA,label = 'LA')
    plt.plot(x,SS_Tau_LeftB,label = 'LB')
    plt.plot(x,SS_Tau_RightA,label = 'RA')
    plt.plot(x,SS_Tau_RightB,label = 'RB')
    plt.title('TorqueCurve')
    plt.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'Torque.jpg')

    plt.figure()
    fig, axs = plt.subplots(2,1)   
    axs[0].plot(x,GRF_yA,label = 'A')
    axs[0].plot(x,GRF_yB,label = 'B')
    axs[1].plot(x,GRF_xA,label = 'A')
    axs[1].plot(x,GRF_xB,label = 'B')
    axs[0].set_title('GRF_Y')
    axs[1].set_title('GRF_X')
    fig.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'GRF_YX.jpg')

    plt.figure()
    fig, axs = plt.subplots(2,1)
    axs[0].plot(x,foot_position_Y_A,label = 'A')
    axs[0].plot(x,foot_position_Y_B,label = 'B')
    axs[1].plot(x,foot_velocity_X_A,label = 'A')
    axs[1].plot(x,foot_velocity_X_B,label = 'B')
    axs[0].set_title('foot_position_Y')
    axs[1].set_title('foot_velocity_X')
    fig.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'Foot_position_velocity.jpg')

    plt.figure()
    plt.plot(x,knee_angle_LA,label = 'LA')
    plt.plot(x,knee_angle_LB,label = 'LB')
    plt.plot(x,knee_angle_RA,label = 'RA')
    plt.plot(x,knee_angle_RB,label = 'RB')
    plt.title('Knee_Angle')
    plt.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'Knee_Angle.jpg')
        
    plt.figure()
    plt.plot(x,dxbody,label = 'dxbody')
    plt.plot(x,dybody,label = 'dybody')
    plt.title('body velocity')
    plt.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'body velocity.jpg') 
        
    plt.figure()  
    plt.plot(x,GRF_yA,label = 'A')
    plt.plot(x,GRF_yB,label = 'B')
    plt.title('GRF_Y')
    plt.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'GRF_Y.jpg') 

    plt.figure()
    plt.plot(x,sfootA,label = 'sfootA')
    plt.plot(x,sfootB,label = 'sfootB')
    plt.title('sfeet')
    plt.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'sfeet.jpg') 

    plt.figure()
    plt.plot(x,foot_position_Y_A,label = 'A')
    plt.plot(x,foot_position_Y_B,label = 'B')
    plt.title('foot_position_Y')
    plt.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'foot_position_Y.jpg') 

    plt.figure()
    plt.plot(x,foot_velocity_X_A,label = 'A')
    plt.plot(x,foot_velocity_X_B,label = 'B')
    plt.title('foot_velocity_X')
    plt.legend()
    if saveGraphs == True:
        plt.savefig(save_folder+'foot_velocity_X.jpg') 


    plt.figure()
    plt.plot(SS_dthULA,SS_Tau_LeftA)
    plt.plot(SS_dthURA,SS_Tau_RightA)
    plt.plot(SS_dthULB,SS_Tau_LeftB)
    plt.plot(SS_dthURB,SS_Tau_RightB)
    plt.grid()
    plt.plot(x_p,rated_torque_p,color = 'green',linestyle = '--')
    plt.plot(x_n,rated_torque_n,color = 'green',linestyle = '--')
    plt.plot(x_p, y_p,'b')
    plt.plot(x_n, stall_torque_p,'b')
    plt.fill_between(x_p, y_p, alpha=0.4, color = 'cyan')
    plt.fill_between(x_n, stall_torque_p, alpha=0.4,color = 'cyan')
    plt.plot(x_n, y_n,'r')
    plt.plot(x_p, stall_torque_n,'r')
    plt.fill_between(x_n, y_n, alpha=0.4,color = 'pink')
    plt.fill_between(x_p, stall_torque_n, alpha=0.4,color = 'pink')
    plt.axis((-30,30,-90,90))
    plt.title('New Torque Speed Curve')
    plt.xlabel('Speed (rad/s)')
    plt.ylabel('Torque (Nm)')
    plt.legend(["Tw_leftA", "Tw_rightA","Tw_leftB", "Tw_rightB","max_torque"], loc=0, frameon=True)
    if saveGraphs == True:
        plt.savefig(save_folder+'TorqueSpeed.jpg')


    fileName = main_name
    f = r"{0}{1}_biped.mp4".format(save_folder,fileName) 
    writervideo = ani.FFMpegWriter(fps=20) 
    animate.save(f, writer=writervideo) 

    print('From here onward (saved files), the angles, velocities, and torques are relative to simscape where CW is positive from the 2.23 rad offset is the origin')    
    ''' Store first values in csv file '''
    import csv
    # csv_file_name = str(int(V_avg))+'_u{0}'.format(mu_value)


    if 'Periodic' in test:
        csv_file_name = save_folder+main_name
        heading0 = ['node_counter','node','collocation_point','hm*h_value','t_steps']
        heading1 = ['x_body','y_body','SS_theta_ULA','SS_theta_URA','SS_theta_ULB','SS_theta_URB']
        heading2 = ['dx_body','dy_body','SS_dtheta_ULA','SS_dtheta_URA','SS_dtheta_ULB','SS_dtheta_URB']
        heading3 = ['ddx_body','ddy_body']
        heading4 = ['SS_Tau_LeftA','SS_Tau_RightA','SS_Tau_LeftB','SS_Tau_RightB', 'GRF_yA','GRF_yB','GRF_xA','GRF_xB']
        heading5 = ['foot_position_Y_A','foot_position_Y_B','foot_position_X_A','foot_position_X_B']
        heading = heading0 + heading1 + heading2 + heading3 + heading4 + heading5

        with open(csv_file_name+'.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(heading)
            t_steps = 0
            node_counter = 0
            
            for n1 in range(1,N+1):
                node = [n1]
                for p in range(0,P):
                    collocation_point = [p]

                    node_counter = node_counter+1
                    
                    # 0% - 50% cycle
                    
                    n = node_counter - 1
                    
                    h_value = [h_values[n]]
                    t_steps = h_values[n] + t_steps
                    data1 = [xbody[n],ybody[n],SS_thULA[n],SS_thURA[n],SS_thULB[n],SS_thURB[n]]
                    data2 = [dxbody[n],dybody[n],SS_dthULA[n],SS_dthURA[n],SS_dthULB[n],SS_dthURB[n]]
                    data3 = [ddxbody[n],ddybody[n]]
                    data4 = [SS_Tau_LeftA[n],SS_Tau_RightA[n],SS_Tau_LeftB[n],SS_Tau_RightB[n],GRF_yA[n],GRF_yB[n],GRF_xA[n],GRF_xB[n]]
                    data5 = [foot_position_Y_A[n],foot_position_Y_B[n],foot_position_X_A[n],foot_position_X_B[n]]
                    data = [node_counter] + node + collocation_point + h_value + [t_steps] + data1 + data2 + data3 + data4 + data5
                    writer.writerow(data)

            for n1 in range(N+1,2*N+1):
                node = [n1]
                for p in range(0,P):
                    collocation_point = [p]
                    node_counter = node_counter+1
             
                    n = node_counter-1-N*P               
                    # 50 - 100% cycle
                    h_value = [h_values[n]]
                    t_steps = h_values[n] + t_steps
                    data1 = [xbody[n],ybody[n],SS_thULB[n],SS_thURB[n],SS_thULA[n],SS_thURA[n]]
                    data2 = [dxbody[n],dybody[n],SS_dthULB[n],SS_dthURB[n],SS_dthULA[n],SS_dthURA[n]]
                    data3 = [ddxbody[n],ddybody[n]]
                    data4 = [SS_Tau_LeftB[n],SS_Tau_RightB[n],SS_Tau_LeftA[n],SS_Tau_RightA[n],GRF_yA[n],GRF_yB[n],GRF_xA[n],GRF_xB[n]]
                    data5 = [foot_position_Y_A[n],foot_position_Y_B[n],foot_position_X_A[n],foot_position_X_B[n]]
                    data =  [node_counter] + node + collocation_point + h_value + [t_steps] + data1 + data2 + data3 + data4 + data5
                    writer.writerow(data)
    
    
    if 'Acceleration' in test:
        csv_file_name = save_folder+main_name
        heading0 = ['node_counter','node','collocation_point','hm*h_value','t_steps']
        heading1 = ['x_body','y_body','SS_theta_ULA','SS_theta_URA','SS_theta_ULB','SS_theta_URB']
        heading2 = ['dx_body','dy_body','SS_dtheta_ULA','SS_dtheta_URA','SS_dtheta_ULB','SS_dtheta_URB']
        heading3 = ['ddx_body','ddy_body']
        heading4 = ['SS_Tau_LeftA','SS_Tau_RightA','SS_Tau_LeftB','SS_Tau_RightB', 'GRF_yA','GRF_yB','GRF_xA','GRF_xB']
        heading5 = ['foot_position_Y_A','foot_position_Y_B','foot_position_X_A','foot_position_X_B']
        heading = heading0 + heading1 + heading2 + heading3 + heading4 + heading5

        with open(csv_file_name+'.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(heading)
            t_steps = 0
            node_counter = 0
            
            for n1 in range(1,N+1):
                node = [n1]
                for p in range(0,P):
                    collocation_point = [p]

                    node_counter = node_counter+1
                    
                    # 0% - 50% cycle
                    
                    n = node_counter - 1
                    
                    h_value = [h_values[n]]
                    t_steps = h_values[n] + t_steps
                    data1 = [xbody[n],ybody[n],SS_thULA[n],SS_thURA[n],SS_thULB[n],SS_thURB[n]]
                    data2 = [dxbody[n],dybody[n],SS_dthULA[n],SS_dthURA[n],SS_dthULB[n],SS_dthURB[n]]
                    data3 = [ddxbody[n],ddybody[n]]
                    data4 = [SS_Tau_LeftA[n],SS_Tau_RightA[n],SS_Tau_LeftB[n],SS_Tau_RightB[n],GRF_yA[n],GRF_yB[n],GRF_xA[n],GRF_xB[n]]
                    data5 = [foot_position_Y_A[n],foot_position_Y_B[n],foot_position_X_A[n],foot_position_X_B[n]]
                    data = [node_counter] + node + collocation_point + h_value + [t_steps] + data1 + data2 + data3 + data4 + data5
                    writer.writerow(data)

    
                    
    with open(csv_file_name+'_Details'+'.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['Test','N','TT','V_avg','cost','mu','h_lower','h_higher','h_sum','tau_sum','pen','Pk Torque','Pk knee angle','Pk GRF Y'])
        writer.writerow([test,N,TT,V_avg,cost_function_str,m.µ.value,h_min,h_max,T,tau_sum,pen_sum,max(SS_Tau_LeftA+SS_Tau_RightA+SS_Tau_LeftB+SS_Tau_RightB,key=abs),max(knee_angle_LA+knee_angle_LB+knee_angle_RA+knee_angle_RB,key=abs),max(GRF_yA+GRF_yB,key=abs)])
else:
    pass

In [ ]:
# hide_sloution('Save Files')
# print('The below save files can only be run for thB =0 as the velocities will not be the same for thB !=0')
# if saveFile == True:
# #     array1 = [['theta_ULA_N1',m.q[1,'theta_ULA'].value*180/np.pi],['theta_ULB_NN',m.q[N,'theta_ULB'].value*180/np.pi]]
# #     array2 = [['theta_ULB_N1',m.q[1,'theta_ULB'].value*180/np.pi],['theta_ULA_NN',m.q[N,'theta_ULA'].value*180/np.pi]]
# #     array3 = [['theta_URA_N1',m.q[1,'theta_URA'].value*180/np.pi],['theta_URB_NN',m.q[N,'theta_URB'].value*180/np.pi]]
# #     array4 = [['theta_URB_N1',m.q[1,'theta_URB'].value*180/np.pi],['theta_URA_NN',m.q[N,'theta_URA'].value*180/np.pi]]

# #     array5 = [['theta_LLA_N1',m.q[1,'theta_LLA'].value*180/np.pi],['theta_LLB_NN',m.q[N,'theta_LLB'].value*180/np.pi]]
# #     array6 = [['theta_LLB_N1',m.q[1,'theta_LLB'].value*180/np.pi],['theta_LLA_NN',m.q[N,'theta_LLA'].value*180/np.pi]]
# #     array7 = [['theta_LRA_N1',m.q[1,'theta_LRA'].value*180/np.pi],['theta_LRB_NN',m.q[N,'theta_LRB'].value*180/np.pi]]
# #     array8 = [['theta_LRB_N1',m.q[1,'theta_LRB'].value*180/np.pi],['theta_LRA_NN',m.q[N,'theta_LRA'].value*180/np.pi]]

# #     array9 = [['dtheta_ULA_N1',m.dq[1,'theta_ULA'].value],['dtheta_ULB_NN',m.dq[N,'theta_ULB'].value]]
# #     array10 = [['dtheta_ULB_N1',m.dq[1,'theta_ULB'].value],['dtheta_ULA_NN',m.dq[N,'theta_ULA'].value]]
# #     array11 = [['dtheta_URA_N1',m.dq[1,'theta_URA'].value],['dtheta_URB_NN',m.dq[N,'theta_URB'].value]]
# #     array12 = [['dtheta_URB_N1',m.dq[1,'theta_URB'].value],['dtheta_URA_NN',m.dq[N,'theta_URA'].value]]

# #     array13 = [['dtheta_LLA_N1',m.dq[1,'theta_LLA'].value],['dtheta_LLB_NN',m.dq[N,'theta_LLB'].value]]
# #     array14 = [['dtheta_LLB_N1',m.dq[1,'theta_LLB'].value],['dtheta_LLA_NN',m.dq[N,'theta_LLA'].value]]
# #     array15 = [['dtheta_LRA_N1',m.dq[1,'theta_LRA'].value],['dtheta_LRB_NN',m.dq[N,'theta_LRB'].value]]
# #     array16 = [['dtheta_LRB_N1',m.dq[1,'theta_LRB'].value],['dtheta_LRA_NN',m.dq[N,'theta_LRA'].value]]

# #     array17 = [['y_bodyN1',m.q[1,'yb'].value],['y_bodyNN',m.q[N,'yb'].value]]
# #     array18 = [['dy_bodyN1',m.dq[1,'yb'].value],['dy_bodyNN',m.dq[N,'yb'].value]]


# #     plt.figure(figsize=(18, 1), dpi=200)
# #     plt.tight_layout()
# #     plt.text(0,1,str(array1))
# #     plt.text(0,2,str(array2))
# #     plt.text(0,3,str(array3))
# #     plt.text(0,4,str(array4))
# #     plt.text(0,5,str(array5))
# #     plt.text(0,6,str(array6))
# #     plt.text(0,7,str(array7))
# #     plt.text(0,8,str(array8))
# #     plt.text(0,9,str(array9))
# #     plt.text(0,10,str(array10))
# #     plt.text(0,11,str(array11))
# #     plt.text(0,12,str(array12))
# #     plt.text(0,13,str(array13))
# #     plt.text(0,14,str(array14))
# #     plt.text(0,15,str(array15))
# #     plt.text(0,16,str(array16))
# #     plt.text(0,17,str(array17))
# #     plt.text(0,18,str(array18))

# #     plt.axis('off')
# #     plt.savefig(save_folder+'array.jpg', bbox_inches='tight')
# #     plt.figure()

#     x = np.arange(1,N*P+1,1)    
#     plt.plot(x,SS_Tau_LeftA,label = 'LA')
#     plt.plot(x,SS_Tau_LeftB,label = 'LB')
#     plt.plot(x,SS_Tau_RightA,label = 'RA')
#     plt.plot(x,SS_Tau_RightB,label = 'RB')
#     plt.title('TorqueCurve')
#     plt.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'Torque.jpg')

#     plt.figure()
#     fig, axs = plt.subplots(2,1)   
#     axs[0].plot(x,GRF_yA,label = 'A')
#     axs[0].plot(x,GRF_yB,label = 'B')
#     axs[1].plot(x,GRF_xA,label = 'A')
#     axs[1].plot(x,GRF_xB,label = 'B')
#     axs[0].set_title('GRF_Y')
#     axs[1].set_title('GRF_X')
#     fig.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'GRF_YX.jpg')

#     plt.figure()
#     fig, axs = plt.subplots(2,1)
#     axs[0].plot(x,foot_position_Y_A,label = 'A')
#     axs[0].plot(x,foot_position_Y_B,label = 'B')
#     axs[1].plot(x,foot_velocity_X_A,label = 'A')
#     axs[1].plot(x,foot_velocity_X_B,label = 'B')
#     axs[0].set_title('foot_position_Y')
#     axs[1].set_title('foot_velocity_X')
#     fig.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'Foot_position_velocity.jpg')

#     plt.figure()
#     plt.plot(x,knee_angle_LA,label = 'LA')
#     plt.plot(x,knee_angle_LB,label = 'LB')
#     plt.plot(x,knee_angle_RA,label = 'RA')
#     plt.plot(x,knee_angle_RB,label = 'RB')
#     plt.title('Knee_Angle')
#     plt.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'Knee_Angle.jpg')
        
#     plt.figure()
#     plt.plot(x,dxbody,label = 'dxbody')
#     plt.plot(x,dybody,label = 'dybody')
#     plt.title('body velocity')
#     plt.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'body velocity.jpg') 
        
#     plt.figure()  
#     plt.plot(x,GRF_yA,label = 'A')
#     plt.plot(x,GRF_yB,label = 'B')
#     plt.title('GRF_Y')
#     plt.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'GRF_Y.jpg') 

#     plt.figure()
#     plt.plot(x,sfootA,label = 'sfootA')
#     plt.plot(x,sfootB,label = 'sfootB')
#     plt.title('sfeet')
#     plt.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'sfeet.jpg') 

#     plt.figure()
#     plt.plot(x,foot_position_Y_A,label = 'A')
#     plt.plot(x,foot_position_Y_B,label = 'B')
#     plt.title('foot_position_Y')
#     plt.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'foot_position_Y.jpg') 

#     plt.figure()
#     plt.plot(x,foot_velocity_X_A,label = 'A')
#     plt.plot(x,foot_velocity_X_B,label = 'B')
#     plt.title('foot_velocity_X')
#     plt.legend()
#     if saveGraphs == True:
#         plt.savefig(save_folder+'foot_velocity_X.jpg') 


#     plt.figure()
#     plt.plot(SS_dthULA,SS_Tau_LeftA)
#     plt.plot(SS_dthURA,SS_Tau_RightA)
#     plt.plot(SS_dthULB,SS_Tau_LeftB)
#     plt.plot(SS_dthURB,SS_Tau_RightB)
#     plt.grid()
#     plt.plot(x_p,rated_torque_p,color = 'green',linestyle = '--')
#     plt.plot(x_n,rated_torque_n,color = 'green',linestyle = '--')
#     plt.plot(x_p, y_p,'b')
#     plt.plot(x_n, stall_torque_p,'b')
#     plt.fill_between(x_p, y_p, alpha=0.4, color = 'cyan')
#     plt.fill_between(x_n, stall_torque_p, alpha=0.4,color = 'cyan')
#     plt.plot(x_n, y_n,'r')
#     plt.plot(x_p, stall_torque_n,'r')
#     plt.fill_between(x_n, y_n, alpha=0.4,color = 'pink')
#     plt.fill_between(x_p, stall_torque_n, alpha=0.4,color = 'pink')
#     plt.axis((-30,30,-90,90))
#     plt.title('New Torque Speed Curve')
#     plt.xlabel('Speed (rad/s)')
#     plt.ylabel('Torque (Nm)')
#     plt.legend(["Tw_leftA", "Tw_rightA","Tw_leftB", "Tw_rightB","rated_torque"], loc=0, frameon=True)
#     if saveGraphs == True:
#         plt.savefig(save_folder+'TorqueSpeed.jpg')



#     loops = 1
#     fileName = main_name
#     f = r"{0}{1}_biped.mp4".format(save_folder,fileName) 
#     writervideo = ani.FFMpegWriter(fps=20) 
#     animate.save(f, writer=writervideo) 

#     print('From here onward (saved files), the angles, velocities, and torques are relative to simscape where CW is positive from the 2.23 rad offset is the origin')    
#     ''' Store first values in csv file '''
#     import csv
#     # csv_file_name = str(int(V_avg))+'_u{0}'.format(mu_value)


#     if 'Periodic' in test:
#         csv_file_name = save_folder+main_name
#         heading0 = ['node','collocation_point','hm*h_value','t_steps']
#         heading1 = ['x_body','y_body','SS_theta_ULA','SS_theta_URA','SS_theta_ULB','SS_theta_URB']
#         heading2 = ['dx_body','dy_body','SS_dtheta_ULA','SS_dtheta_URA','SS_dtheta_ULB','SS_dtheta_URB']
#         heading3 = ['ddx_body','ddy_body']
#         heading4 = ['SS_Tau_LeftA','SS_Tau_RightA','SS_Tau_LeftB','SS_Tau_RightB', 'GRF_yA','GRF_yB','GRF_xA','GRF_xB']
#         heading5 = ['foot_position_Y_A','foot_position_Y_B','foot_position_X_A','foot_position_X_B']
#         heading = heading0 + heading1 + heading2 + heading3 + heading4 + heading5

#         with open(csv_file_name+'.csv', 'w') as f:
#             writer = csv.writer(f)
#             writer.writerow(heading)
#             t_steps = 0
#             for repeat in range(0,loops): 
#                 for n in range(2*N*P*repeat,N*P*(2*repeat+1)):
#                     node = [n+1]
#                     n = n+1-repeat*N*P*2-1
#                     # 0 - 50 cycle
#                     h_value = [h_values[n]]
#                     t_steps = h_values[n] + t_steps
#                     data1 = [xbody[n],ybody[n],SS_thULA[n],SS_thURA[n],SS_thULB[n],SS_thURB[n]]
#                     data2 = [dxbody[n],dybody[n],SS_dthULA[n],SS_dthURA[n],SS_dthULB[n],SS_dthURB[n]]
#                     data3 = [ddxbody[n],ddybody[n]]
#                     data4 = [SS_Tau_LeftA[n],SS_Tau_RightA[n],SS_Tau_LeftB[n],SS_Tau_RightB[n],GRF_yA[n],GRF_yB[n],GRF_xA[n],GRF_xB[n]]
#                     data5 = [foot_position_Y_A[n],foot_position_Y_B[n],foot_position_X_A[n],foot_position_X_B[n]]
#                     data = node + h_value + [t_steps] + data1 + data2 + data3 + data4 + data5
#                     writer.writerow(data)

#                 for n in range(N*P*(2*repeat+1),2*N*P*(repeat+1)):
#                     node = [n+1]
#                     n = n+1-repeat*N*P*2-N*P-1
#                     # 50 - 100 cycle
#                     h_value = [h_values[n]]
#                     t_steps = h_values[n] + t_steps
#                     data1 = [xbody[n],ybody[n],SS_thULB[n],SS_thURB[n],SS_thULA[n],SS_thURA[n]]
#                     data2 = [dxbody[n],dybody[n],SS_dthULB[n],SS_dthURB[n],SS_dthULA[n],SS_dthURA[n]]
#                     data3 = [ddxbody[n],ddybody[n]]
#                     data4 = [SS_Tau_LeftB[n],SS_Tau_RightB[n],SS_Tau_LeftA[n],SS_Tau_RightA[n],GRF_yA[n],GRF_yB[n],GRF_xA[n],GRF_xB[n]]
#                     data5 = [foot_position_Y_A[n],foot_position_Y_B[n],foot_position_X_A[n],foot_position_X_B[n]]
#                     data = node + h_value + [t_steps] + data1 + data2 + data3 + data4 + data5
#                     writer.writerow(data)
    
#     else:
#         csv_file_name = save_folder+main_name
#         heading0 = ['node','hm*h_value','t_steps']
#         heading1 = ['x_body','y_body','SS_theta_ULA','SS_theta_URA','SS_theta_ULB','SS_theta_URB']
#         heading2 = ['dx_body','dy_body','SS_dtheta_ULA','SS_dtheta_URA','SS_dtheta_ULB','SS_dtheta_URB']
#         heading3 = ['ddx_body','ddy_body']
#         heading4 = ['SS_Tau_LeftA','SS_Tau_RightA','SS_Tau_LeftB','SS_Tau_RightB', 'GRF_yA','GRF_yB','GRF_xA','GRF_xB']
#         heading5 = ['foot_position_Y_A','foot_position_Y_B','foot_position_X_A','foot_position_X_B']
#         heading = heading0 + heading1 + heading2 + heading3 + heading4 + heading5

#         with open(csv_file_name+'.csv', 'w') as f:
#             writer = csv.writer(f)
#             writer.writerow(heading)
#             t_steps = 0
#             for repeat in range(0,loops): 
#                 for n in range(2*N*repeat,N*(2*repeat+1)):
#                     node = [n+1]
#                     n = n+1-repeat*N*2-1
#                     # 0 - 50 cycle
#                     h_value = [h_values[n]]
#                     t_steps = h_values[n] + t_steps
#                     data1 = [xbody[n],ybody[n],SS_thULA[n],SS_thURA[n],SS_thULB[n],SS_thURB[n]]
#                     data2 = [dxbody[n],dybody[n],SS_dthULA[n],SS_dthURA[n],SS_dthULB[n],SS_dthURB[n]]
#                     data3 = [ddxbody[n],ddybody[n]]
#                     data4 = [SS_Tau_LeftA[n],SS_Tau_RightA[n],SS_Tau_LeftB[n],SS_Tau_RightB[n],GRF_yA[n],GRF_yB[n],GRF_xA[n],GRF_xB[n]]
#                     data5 = [foot_position_Y_A[n],foot_position_Y_B[n],foot_position_X_A[n],foot_position_X_B[n]]
#                     data = node + h_value + [t_steps] + data1 + data2 + data3 + data4 + data5
#                     writer.writerow(data)
                    
#     with open(csv_file_name+'_Details'+'.csv', 'w') as f:
#         writer = csv.writer(f)
#         writer.writerow(['Test','N','TT','V_avg','loops','cost','mu','h_lower','h_higher','h_sum','tau_sum','pen','Pk Torque','Pk knee angle','Pk GRF Y'])
#         writer.writerow([test,N,TT,V_avg,loops,cost_function_str,m.µ.value,h_min,h_max,T,tau_sum,pen_sum,max(SS_Tau_LeftA+SS_Tau_RightA+SS_Tau_LeftB+SS_Tau_RightB,key=abs),max(knee_angle_LA+knee_angle_LB+knee_angle_RA+knee_angle_RB,key=abs),max(GRF_yA+GRF_yB,key=abs)])
# else:
#     pass

In [ ]:
print('sum_h = ',T)

print('N = ',N)
print('cost = ',cost_function_str)


print('V_avg',V_avg)
print('TT = ',TT)
print('tau_sum = ',tau_sum)
print('Pk Torque = ',max(SS_Tau_LeftA+SS_Tau_RightA+SS_Tau_LeftB+SS_Tau_RightB,key=abs))
print('Pk knee angle = ',max(knee_angle_LA+knee_angle_LB+knee_angle_RA+knee_angle_RB,key=abs))
print('Pk GRF Y = ',max(GRF_yA+GRF_yB,key=abs))

In [ ]:
plt.figure()  
plt.plot(x,GRF_yA,label = 'A')
plt.plot(x,GRF_yB,label = 'B')
plt.title('GRF_Y')
plt.legend()

plt.figure()
plt.plot(x,sfootA,label = 'sfootA')
plt.plot(x,sfootB,label = 'sfootB')
plt.title('sfeet')
plt.legend()

plt.figure()
plt.plot(x,foot_position_Y_A,label = 'A')
plt.plot(x,foot_position_Y_B,label = 'B')
plt.title('foot_position_Y')
plt.legend()

plt.figure()
plt.plot(x,foot_velocity_X_A,label = 'A')
plt.plot(x,foot_velocity_X_B,label = 'B')
plt.title('foot_velocity_X')
plt.legend()

In [ ]:
for n in range(1,N+1):
    for leg in legs:
        print(m.ground_penalty[n,'slip_ps',leg].value)